## Este Jupyter notebook Irá realizar a análise de imagens de pokemnos utilizando CNN's para obtermos seus atributos e com isso sermos capazes de definir quem venceria em uma RINHA POKEMÓN

<img src="https://camo.githubusercontent.com/4d6196ed859ab0239ec09ce7ae79c7e8a4bad20e0e8ecbecdd31f26c6e612c50/68747470733a2f2f75706c6f61642e77696b696d656469612e6f72672f77696b6970656469612f636f6d6d6f6e732f7468756d622f392f39372f506f6b656d6f6e5f547970655f43686172742e7376672f3130323470782d506f6b656d6f6e5f547970655f43686172742e7376672e706e67" width="800"/>



A tabela acima representa as forças e fraquezas que serão utilizadas como regras de batalha para definir os vencedores baseados em seus atributos

In [2]:
# Importando bibliotecas
import os
import requests
from IPython.display import display, HTML, clear_output
import pandas as pd
import time
from tqdm.notebook import tqdm
from loguru import logger

Aqui iremos definir algumas funções e classes para obtenção dos nossos dados

In [3]:
def get_pokemon(pokemon_id: int):
    url = f"https://pokeapi.co/api/v2/pokemon/{pokemon_id}/"
    resp = requests.get(url)
    resp.raise_for_status()
    data = resp.json()
    # Use official artwork (HD ~475x475px)
    hd_url = data["sprites"]["other"]["official-artwork"]["front_default"]
    return {
        "id": data["id"],
        "name": data["name"].title(),
        "species": data["species"]["name"],
        "weight": data["weight"],
        "types": [t["type"]["name"] for t in data["types"]],
        "image_url": hd_url,
        "abilities": [a["ability"]["name"] for a in data["abilities"]],
        "stats": {s["stat"]["name"]: s["base_stat"] for s in data["stats"]},
        "moves": [m["move"]["name"] for m in data["moves"]],
    }

def show_pokemon(pokemon_id: int, width: int = 200, max_moves: int = 10):
    p = get_pokemon(pokemon_id)
    # show only up to max_moves
    moves_txt = ", ".join(p['moves'][:max_moves]) + ("..." if len(p['moves']) > max_moves else "")
    stats_rows = "".join(f"<tr><th align='left'>{name.title()}</th><td>{val}</td></tr>"
                         for name, val in p["stats"].items())
    html = f"""
    <div style="display:flex; align-items:flex-start; font-family: Arial, sans-serif;">
      <img src="{p['image_url']}" width="{width}px" alt="{p['name']}"/>
      <div style="margin-left:16px;">
        <h2>#{p['id']} — {p['name']}</h2>
        <p><strong>Species:</strong> {p['species'].title()}</p>
        <p><strong>Weight:</strong> {p['weight']}</p>
        <p><strong>Types:</strong> {', '.join(t.title() for t in p['types'])}</p>
        <p><strong>Abilities:</strong> {', '.join(a.title() for a in p['abilities'])}</p>
        <h4>Base Stats</h4>
        <table>
          {stats_rows}
        </table>
        <p><strong>Moves:</strong> {moves_txt}</p>
      </div>
    </div>
    """
    display(HTML(html))

def pokemon_to_df(pokemon_data: dict) -> pd.DataFrame:
    # Flatten the data for a single row
    row = {
        'id': pokemon_data['id'],
        'name': pokemon_data['name'],
        'species': pokemon_data['species'],
        'weight': pokemon_data['weight'],
        'types': ', '.join(pokemon_data['types']),
        'abilities': ', '.join(pokemon_data['abilities']),
        'image_url': pokemon_data['image_url'],
        'moves': ', '.join(pokemon_data['moves']),
    }
    
    # Add stats as individual columns
    for stat_name, stat_value in pokemon_data['stats'].items():
        row[f'stat_{stat_name}'] = stat_value
    
    # Create DataFrame with a single row
    df = pd.DataFrame([row])
    
    return df

def pokemon_to_df_row(pokemon_data: dict) -> dict:
    row = {
        'id': pokemon_data['id'],
        'name': pokemon_data['name'],
        'species': pokemon_data['species'],
        'weight': pokemon_data['weight'],
        'types': ', '.join(pokemon_data['types']),
        'abilities': ', '.join(pokemon_data['abilities']),
        'image_url': pokemon_data['image_url'],
        'moves': ', '.join(pokemon_data['moves']),
    }
    for stat_name, stat_value in pokemon_data['stats'].items():
        row[f'stat_{stat_name}'] = stat_value
    return row

# Loop through IDs and collect data
def collect_pokemon_data(start_id=1, end_id=1025, delay=1):
    rows = []
    for pid in range(start_id, end_id + 1):
        try:
            poke_data = get_pokemon(pid)
            row = pokemon_to_df_row(poke_data)
            rows.append(row)
            logger.info(f"Fetched Pokémon ID: {pid} ({poke_data['name']}) — Total collected: {len(rows)}")
        except Exception as e:
            logger.error(f"Failed for ID {pid}: {e}")
        time.sleep(delay)  # be nice to the API!
    df = pd.DataFrame(rows)
    return df

def download_pokemon_images(df, output_dir='train_set'):
    os.makedirs(output_dir, exist_ok=True)

    for row in tqdm(df.itertuples(), total=len(df), desc="Downloading HD images"):
        image_url = getattr(row, 'image_url')
        name = getattr(row, 'name').lower()
        pid = getattr(row, 'id')
        if not image_url:
            logger.warning(f"No image available for {name}_{pid}")
            continue

        filename = f"{name}_{pid}.png"
        filepath = os.path.join(output_dir, filename)

        try:
            resp = requests.get(image_url, timeout=10)
            resp.raise_for_status()
            with open(filepath, 'wb') as f:
                f.write(resp.content)
        except Exception as e:
            logger.error(f"❌ Failed to download {filename}: {e}")


Iremos treinar nossa CNN utilizando os pokemons pós primeira geração do 152 ao 1015 pokemon's

In [75]:
show_pokemon(152, width=400, max_moves=8)

Hp,45
Attack,49
Defense,65
Special-Attack,49
Special-Defense,65
Speed,45


In [76]:
# Example: display Pikachu (ID 25)
show_pokemon(1025, width=400, max_moves=8)

Hp,88
Attack,88
Defense,160
Special-Attack,88
Special-Defense,88
Speed,88


Note que estamos obtendo as fotos oficiais em HD dos pokemons que nos provê imagens de 475x475 pixels (aproximadamente)

In [77]:
example = get_pokemon(152)

vamos dar um olhada de como está o retorno do noss json 

In [79]:
example

{'id': 152,
 'name': 'Chikorita',
 'species': 'chikorita',
 'weight': 64,
 'types': ['grass'],
 'image_url': 'https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/other/official-artwork/152.png',
 'abilities': ['overgrow', 'leaf-guard'],
 'stats': {'hp': 45,
  'attack': 49,
  'defense': 65,
  'special-attack': 49,
  'special-defense': 65,
  'speed': 45},
 'moves': ['swords-dance',
  'cut',
  'vine-whip',
  'headbutt',
  'tackle',
  'body-slam',
  'take-down',
  'double-edge',
  'growl',
  'counter',
  'leech-seed',
  'razor-leaf',
  'solar-beam',
  'poison-powder',
  'toxic',
  'mimic',
  'double-team',
  'light-screen',
  'reflect',
  'flash',
  'rest',
  'substitute',
  'snore',
  'curse',
  'flail',
  'protect',
  'mud-slap',
  'detect',
  'giga-drain',
  'endure',
  'charm',
  'swagger',
  'fury-cutter',
  'attract',
  'sleep-talk',
  'return',
  'frustration',
  'safeguard',
  'encore',
  'sweet-scent',
  'iron-tail',
  'synthesis',
  'hidden-power',
  'sunny-d

Agora iremos transformar isto em um dataframe para avaliarmos nossos dados e refletirmos sobre como iremos trabalhar com os mesmos 

In [80]:
df_pokemon = collect_pokemon_data(152, 1025)

2025-06-24 21:49:30.738 | INFO     | __main__:collect_pokemon_data:90 - Fetched Pokémon ID: 152 (Chikorita) — Total collected: 1
2025-06-24 21:49:32.109 | INFO     | __main__:collect_pokemon_data:90 - Fetched Pokémon ID: 153 (Bayleef) — Total collected: 2
2025-06-24 21:49:33.508 | INFO     | __main__:collect_pokemon_data:90 - Fetched Pokémon ID: 154 (Meganium) — Total collected: 3
2025-06-24 21:49:34.906 | INFO     | __main__:collect_pokemon_data:90 - Fetched Pokémon ID: 155 (Cyndaquil) — Total collected: 4
2025-06-24 21:49:36.314 | INFO     | __main__:collect_pokemon_data:90 - Fetched Pokémon ID: 156 (Quilava) — Total collected: 5
2025-06-24 21:49:37.705 | INFO     | __main__:collect_pokemon_data:90 - Fetched Pokémon ID: 157 (Typhlosion) — Total collected: 6
2025-06-24 21:49:39.178 | INFO     | __main__:collect_pokemon_data:90 - Fetched Pokémon ID: 158 (Totodile) — Total collected: 7
2025-06-24 21:49:40.694 | INFO     | __main__:collect_pokemon_data:90 - Fetched Pokémon ID: 159 (Croco

In [84]:
df_pokemon.head(15)

,id,name,species,weight,types,abilities,image_url,moves,stat_hp,stat_attack,stat_defense,stat_special-attack,stat_special-defense,stat_speed
0,152,Chikorita,chikorita,64,grass,"overgrow, leaf-guard",https://raw.githubusercontent.com/PokeAPI/spri...,"swords-dance, cut, vine-whip, headbutt, tackle...",45,49,65,49,65,45
1,153,Bayleef,bayleef,158,grass,"overgrow, leaf-guard",https://raw.githubusercontent.com/PokeAPI/spri...,"swords-dance, cut, vine-whip, headbutt, tackle...",60,62,80,63,80,60
2,154,Meganium,meganium,1005,grass,"overgrow, leaf-guard",https://raw.githubusercontent.com/PokeAPI/spri...,"swords-dance, cut, vine-whip, headbutt, tackle...",80,82,100,83,100,80
3,155,Cyndaquil,cyndaquil,79,fire,"blaze, flash-fire",https://raw.githubusercontent.com/PokeAPI/spri...,"cut, double-kick, headbutt, tackle, body-slam,...",39,52,43,60,50,65
4,156,Quilava,quilava,190,fire,"blaze, flash-fire",https://raw.githubusercontent.com/PokeAPI/spri...,"cut, double-kick, headbutt, tackle, body-slam,...",58,64,58,80,65,80
5,157,Typhlosion,typhlosion,795,fire,"blaze, flash-fire",https://raw.githubusercontent.com/PokeAPI/spri...,"mega-punch, fire-punch, thunder-punch, cut, do...",78,84,78,109,85,100
6,158,Totodile,totodile,95,water,"torrent, sheer-force",https://raw.githubusercontent.com/PokeAPI/spri...,"mega-punch, ice-punch, scratch, razor-wind, sw...",50,65,64,44,48,43
7,159,Croconaw,croconaw,250,water,"torrent, sheer-force",https://raw.githubusercontent.com/PokeAPI/spri...,"mega-punch, ice-punch, scratch, swords-dance, ...",65,80,80,59,63,58
8,160,Feraligatr,feraligatr,888,water,"torrent, sheer-force",https://raw.githubusercontent.com/PokeAPI/spri...,"mega-punch, ice-punch, scratch, swords-dance, ...",85,105,100,79,83,78
9,161,Sentret,sentret,60,normal,"run-away, keen-eye, frisk",https://raw.githubusercontent.com/PokeAPI/spri...,"fire-punch, ice-punch, thunder-punch, scratch,...",35,46,34,35,45,20


In [ ]:
# save results as csv
df_pokemon.to_csv('train_set/pokemon_data.csv', index=False)
# read from csv in the future to avoid long process time
# df_loaded = pd.read_csv('train_set/pokemon_data.csv')

In [4]:
df_pokemon = pd.read_csv('train_set/pokemon_data.csv')

Agora vamos realizar o download de nossas imagens para construir nosso conjunto de treino 

In [83]:
# Realizando download das imagens obtidas pela url do dataframe
download_pokemon_images(df_pokemon)


Agora obtemos nossos dados vamos definir as principais features para que possamos mapear as informações que queremos extrair da imagem afim de descobrir o tipo de nosso pokemon

In [5]:
import numpy as np
import skimage.io
import skimage.color
import skimage.measure


In [6]:
def extract_image_features(image_path):
    image = skimage.io.imread(image_path)

    # Handle RGB or RGBA
    if image.shape[-1] == 4:
        rgb = image[..., :3]
        alpha = image[..., 3] / 255.0
    else:
        rgb = image
        alpha = np.ones(rgb.shape[:2])

    # Grayscale from RGB
    rgb_gray = skimage.color.rgb2gray(rgb)

    # Binary mask for region detection (ignore transparency)
    thresh = rgb_gray > 0.1
    labeled = skimage.measure.label(thresh)
    regions = skimage.measure.regionprops(labeled, intensity_image=rgb_gray)

    if len(regions) == 0:
        return {  # Return zeros with names
            'region_area': 0, 'region_perimeter': 0, 'region_eccentricity': 0,
            'region_solidity': 0, 'region_extent': 0, 'region_circularity': 0,
            'region_aspect_ratio': 0, 'region_euler_number': 0,
            'intensity_mean': 0, 'intensity_std': 0, 'intensity_min': 0, 'intensity_max': 0,
            'intensity_median': 0, 'intensity_q25': 0, 'intensity_q75': 0,
            'color_r_mean': 0, 'color_g_mean': 0, 'color_b_mean': 0,
            'color_r_std': 0, 'color_g_std': 0, 'color_b_std': 0,
            'alpha_mean': 0, 'alpha_std': 0, 'alpha_min': 0, 'alpha_max': 0
        }

    region = max(regions, key=lambda r: r.area)

    # Shape features
    shape_features = {
        'region_area': region.area,
        'region_perimeter': region.perimeter,
        'region_eccentricity': region.eccentricity,
        'region_solidity': region.solidity,
        'region_extent': region.extent,
        'region_circularity': region.perimeter / (4 * np.sqrt(region.area)) if region.area > 0 else 0,
        'region_aspect_ratio': region.major_axis_length / (region.minor_axis_length + 1e-10),
        'region_euler_number': region.euler_number
    }

    # Intensity
    coords = region.coords
    intensities = rgb_gray[coords[:, 0], coords[:, 1]]
    intensity_features = {
        'intensity_mean': intensities.mean(),
        'intensity_std': intensities.std() if len(intensities) > 1 else 0,
        'intensity_min': np.min(intensities),
        'intensity_max': np.max(intensities),
        'intensity_median': np.median(intensities),
        'intensity_q25': np.percentile(intensities, 25),
        'intensity_q75': np.percentile(intensities, 75)
    }

    # Colors
    region_mask = labeled == region.label
    region_pixels = rgb[region_mask]
    color_means = region_pixels.mean(axis=0) if region_pixels.size else [0, 0, 0]
    color_stds = region_pixels.std(axis=0) if region_pixels.size else [0, 0, 0]
    color_features = {
        'color_r_mean': color_means[0],
        'color_g_mean': color_means[1],
        'color_b_mean': color_means[2],
        'color_r_std': color_stds[0],
        'color_g_std': color_stds[1],
        'color_b_std': color_stds[2],
    }

    # Alpha
    alpha_pixels = alpha[region_mask]
    alpha_features = {
        'alpha_mean': alpha_pixels.mean() if alpha_pixels.size else 0,
        'alpha_std': alpha_pixels.std() if alpha_pixels.size else 0,
        'alpha_min': alpha_pixels.min() if alpha_pixels.size else 0,
        'alpha_max': alpha_pixels.max() if alpha_pixels.size else 0,
    }

    # Combine all
    features = {}
    features.update(shape_features)
    features.update(intensity_features)
    features.update(color_features)
    features.update(alpha_features)

    return features


In [27]:
# exemplo de extração de uma feature
extract_image_features('C:/Users/Admin/Desktop/Projetos_Pessoais/Mestrado/Deep_Learning/Atividades/Trabalho_final_pokemon/Pokemon-AI-Battle/back-end/train_set/absol_359.png')

{'region_area': 151465.0,
 'region_perimeter': 7464.088850160816,
 'region_eccentricity': 0.48425826857056925,
 'region_solidity': 0.6713130193905817,
 'region_extent': 0.6713130193905817,
 'region_circularity': 4.794691450516671,
 'region_aspect_ratio': 1.1429549807654842,
 'region_euler_number': -347,
 'intensity_mean': 0.9292581433334431,
 'intensity_std': 0.17253357654620083,
 'intensity_min': 0.10007098039215687,
 'intensity_max': 1.0,
 'intensity_median': 1.0,
 'intensity_q25': 0.9756372549019608,
 'intensity_q75': 1.0,
 'color_r_mean': 235.20433763575744,
 'color_g_mean': 237.0478328326676,
 'color_b_mean': 241.2744132307794,
 'color_r_std': 47.51517089340414,
 'color_g_std': 43.85028866500044,
 'color_b_std': 36.17882465526285,
 'alpha_mean': 0.3291376833967337,
 'alpha_std': 0.4698366099406737,
 'alpha_min': 0.0,
 'alpha_max': 1.0}

In [7]:
def process_images_in_folder(folder_path):
    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    records = []

    for fname in tqdm(image_files, desc="Processing images"):
        full_path = os.path.join(folder_path, fname)

        # Extract ID from filename (expects format name_id.png)
        try:
            base = os.path.splitext(fname)[0]
            parts = base.split('_')
            pokemon_id = int(parts[-1])
        except Exception as e:
            logger.error(f"⚠️ Skipping invalid filename: {fname}")
            continue

        # Extract features
        try:
            features = extract_image_features(full_path)
            row = {
                'image_path': full_path,
                'id': pokemon_id,
                **features
            }
            records.append(row)
            logger.info(f'✅ Processed {fname}')
        except Exception as e:
            logger.error(f"❌ Error processing {fname}: {e}")
            continue

    return pd.DataFrame(records)

In [110]:
df_pokemon_features = process_images_in_folder('train_set')

Processing images:   0%|          | 0/874 [00:00<?, ?it/s]

2025-06-25 01:44:33.428 | INFO     | __main__:process_images_in_folder:26 - ✅ Processed abomasnow_460.png
2025-06-25 01:44:33.534 | INFO     | __main__:process_images_in_folder:26 - ✅ Processed absol_359.png
2025-06-25 01:44:33.603 | INFO     | __main__:process_images_in_folder:26 - ✅ Processed accelgor_617.png
2025-06-25 01:44:33.659 | INFO     | __main__:process_images_in_folder:26 - ✅ Processed aegislash-shield_681.png
2025-06-25 01:44:33.723 | INFO     | __main__:process_images_in_folder:26 - ✅ Processed aggron_306.png
2025-06-25 01:44:33.786 | INFO     | __main__:process_images_in_folder:26 - ✅ Processed aipom_190.png
2025-06-25 01:44:33.868 | INFO     | __main__:process_images_in_folder:26 - ✅ Processed alcremie_869.png
2025-06-25 01:44:33.929 | INFO     | __main__:process_images_in_folder:26 - ✅ Processed alomomola_594.png
2025-06-25 01:44:33.997 | INFO     | __main__:process_images_in_folder:26 - ✅ Processed altaria_334.png
2025-06-25 01:44:34.080 | INFO     | __main__:process_

In [8]:
#df_pokemon_features.head()

In [112]:
df_pokemon_features_final = pd.merge(df_pokemon_features, df_pokemon[['id', 'name', 'types']], on='id', how='left')


In [113]:
df_pokemon_features_final.head()

,image_path,id,region_area,region_perimeter,region_eccentricity,region_solidity,region_extent,region_circularity,region_aspect_ratio,region_euler_number,...,color_b_mean,color_r_std,color_g_std,color_b_std,alpha_mean,alpha_std,alpha_min,alpha_max,name,types
0,train_set\abomasnow_460.png,460,107151.0,7448.593826,0.371940,0.685367,0.584940,5.688741,1.077288,-946,...,194.553807,70.389289,58.600939,58.962556,0.999971,0.001373,0.862745,1.0,Abomasnow,"grass, ice"
1,train_set\absol_359.png,359,151465.0,7464.088850,0.484258,0.671313,0.671313,4.794691,1.142955,-347,...,241.274413,47.515171,43.850289,36.178825,0.329138,0.469837,0.000000,1.0,Absol,dark
2,train_set\accelgor_617.png,617,122602.0,8408.956995,0.310894,0.543388,0.543388,6.003899,1.052139,-552,...,190.481844,72.049311,72.536374,67.004721,0.528052,0.499164,0.000000,1.0,Accelgor,bug
3,train_set\aegislash-shield_681.png,681,71327.0,3475.656854,0.534266,0.316131,0.316131,3.253493,1.182989,0,...,255.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,Aegislash-Shield,"steel, ghost"
4,train_set\aggron_306.png,306,147550.0,15195.167779,0.487770,0.653961,0.653961,9.889536,1.145512,-807,...,169.569143,76.058984,75.476941,73.497089,0.725863,0.446061,0.000000,1.0,Aggron,"steel, rock"


In [114]:
df_pokemon_features_final.shape

(874, 29)

In [116]:
df_pokemon_features_final.to_csv("train_set/pokemon_image_features.csv", index=False)

In [8]:
df_pokemon_features_final = pd.read_csv("train_set/pokemon_image_features.csv")

In [9]:
from sklearn.preprocessing import MultiLabelBinarizer

def one_hot_encode_types(df, type_col="types"):
    df = df.copy()
    # Split comma-separated string into list
    df["type_list"] = df[type_col].str.split(",\s*")
    mlb = MultiLabelBinarizer()
    type_encoded = mlb.fit_transform(df["type_list"])
    type_df = pd.DataFrame(type_encoded, columns=mlb.classes_)
    df = pd.concat([df.drop(columns=[type_col, "type_list"]), type_df], axis=1)
    return df


In [10]:
df_pokemon_features_final2 = one_hot_encode_types(df_pokemon_features_final, type_col="types")

In [11]:
df_pokemon_features_final2.head(4)

,image_path,id,region_area,region_perimeter,region_eccentricity,region_solidity,region_extent,region_circularity,region_aspect_ratio,region_euler_number,...,ghost,grass,ground,ice,normal,poison,psychic,rock,steel,water
0,train_set\abomasnow_460.png,460,107151.0,7448.593826,0.371940,0.685367,0.584940,5.688741,1.077288,-946,...,0,1,0,1,0,0,0,0,0,0
1,train_set\absol_359.png,359,151465.0,7464.088850,0.484258,0.671313,0.671313,4.794691,1.142955,-347,...,0,0,0,0,0,0,0,0,0,0
2,train_set\accelgor_617.png,617,122602.0,8408.956995,0.310894,0.543388,0.543388,6.003899,1.052139,-552,...,0,0,0,0,0,0,0,0,0,0
3,train_set\aegislash-shield_681.png,681,71327.0,3475.656854,0.534266,0.316131,0.316131,3.253493,1.182989,0,...,1,0,0,0,0,0,0,0,1,0


In [90]:
df_pokemon_features_final2.to_csv("train_set/pokemon_image_features_ohe.csv", index=False)

Agora vamos para a principal parte do nosso código a criação e treinamento de nossa CNN

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader, Dataset
from sklearn.preprocessing import StandardScaler
import torch.optim as optim
import cv2

In [14]:
torch.device("cuda" if torch.cuda.is_available() else "cpu")

device(type='cuda')

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingLR
from PIL import Image
import pandas as pd
import numpy as np
import skimage
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
import os
from loguru import logger
import time
import gc

In [15]:
# Configure loguru logger
logger.add("logs/pokemon_training_{time}.log", 
           format="{time:YYYY-MM-DD HH:mm:ss} | {level} | {message}",
           level="INFO")
logger.add(lambda msg: print(msg, end=""), 
           format="{time:HH:mm:ss} | {level} | {message}",
           level="INFO", colorize=True)

# Create directories if they don't exist
os.makedirs("models", exist_ok=True)
os.makedirs("logs", exist_ok=True)

# Keep your original function exactly as is
def extract_image_features(image_path):
    image = skimage.io.imread(image_path)

    # Handle RGB or RGBA
    if image.shape[-1] == 4:
        rgb = image[..., :3]
        alpha = image[..., 3] / 255.0
    else:
        rgb = image
        alpha = np.ones(rgb.shape[:2])

    # Grayscale from RGB
    rgb_gray = skimage.color.rgb2gray(rgb)

    # Binary mask for region detection (ignore transparency)
    thresh = rgb_gray > 0.1
    labeled = skimage.measure.label(thresh)
    regions = skimage.measure.regionprops(labeled, intensity_image=rgb_gray)

    if len(regions) == 0:
        return {  # Return zeros with names
            'region_area': 0, 'region_perimeter': 0, 'region_eccentricity': 0,
            'region_solidity': 0, 'region_extent': 0, 'region_circularity': 0,
            'region_aspect_ratio': 0, 'region_euler_number': 0,
            'intensity_mean': 0, 'intensity_std': 0, 'intensity_min': 0, 'intensity_max': 0,
            'intensity_median': 0, 'intensity_q25': 0, 'intensity_q75': 0,
            'color_r_mean': 0, 'color_g_mean': 0, 'color_b_mean': 0,
            'color_r_std': 0, 'color_g_std': 0, 'color_b_std': 0,
            'alpha_mean': 0, 'alpha_std': 0, 'alpha_min': 0, 'alpha_max': 0
        }

    region = max(regions, key=lambda r: r.area)

    # Shape features
    shape_features = {
        'region_area': region.area,
        'region_perimeter': region.perimeter,
        'region_eccentricity': region.eccentricity,
        'region_solidity': region.solidity,
        'region_extent': region.extent,
        'region_circularity': region.perimeter / (4 * np.sqrt(region.area)) if region.area > 0 else 0,
        'region_aspect_ratio': region.major_axis_length / (region.minor_axis_length + 1e-10),
        'region_euler_number': region.euler_number
    }

    # Intensity
    coords = region.coords
    intensities = rgb_gray[coords[:, 0], coords[:, 1]]
    intensity_features = {
        'intensity_mean': intensities.mean(),
        'intensity_std': intensities.std() if len(intensities) > 1 else 0,
        'intensity_min': np.min(intensities),
        'intensity_max': np.max(intensities),
        'intensity_median': np.median(intensities),
        'intensity_q25': np.percentile(intensities, 25),
        'intensity_q75': np.percentile(intensities, 75)
    }

    # Colors
    region_mask = labeled == region.label
    region_pixels = rgb[region_mask]
    color_means = region_pixels.mean(axis=0) if region_pixels.size else [0, 0, 0]
    color_stds = region_pixels.std(axis=0) if region_pixels.size else [0, 0, 0]
    color_features = {
        'color_r_mean': color_means[0],
        'color_g_mean': color_means[1],
        'color_b_mean': color_means[2],
        'color_r_std': color_stds[0],
        'color_g_std': color_stds[1],
        'color_b_std': color_stds[2],
    }

    # Alpha
    alpha_pixels = alpha[region_mask]
    alpha_features = {
        'alpha_mean': alpha_pixels.mean() if alpha_pixels.size else 0,
        'alpha_std': alpha_pixels.std() if alpha_pixels.size else 0,
        'alpha_min': alpha_pixels.min() if alpha_pixels.size else 0,
        'alpha_max': alpha_pixels.max() if alpha_pixels.size else 0,
    }

    # Combine all
    features = {}
    features.update(shape_features)
    features.update(intensity_features)
    features.update(color_features)
    features.update(alpha_features)

    return features

# Optimized CNN model with improved architecture
class PokemonCNN(nn.Module):
    def __init__(self, num_classes=18, num_handcrafted_features=26):
        super(PokemonCNN, self).__init__()
        
        logger.info(f"Initializing PokemonCNN with {num_classes} classes and {num_handcrafted_features} handcrafted features")
        
        # Enhanced CNN feature extractor with residual connections
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(512)
        self.conv5 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.bn5 = nn.BatchNorm2d(512)
        
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout2d = nn.Dropout2d(0.2)
        
        # Adaptive pooling to handle different input sizes
        self.adaptive_pool = nn.AdaptiveAvgPool2d((4, 4))
        
        # Enhanced fully connected layers for CNN features
        self.fc_cnn = nn.Sequential(
            nn.Linear(512 * 4 * 4, 1024),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(1024),
            nn.Dropout(0.5),
            nn.Linear(1024, 512),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(512),
            nn.Dropout(0.3)
        )
        
        # Enhanced fully connected layers for handcrafted features
        self.fc_handcrafted = nn.Sequential(
            nn.Linear(num_handcrafted_features, 256),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(256),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(128),
            nn.Dropout(0.2)
        )
        
        # Enhanced combined classifier with attention mechanism
        self.attention = nn.Sequential(
            nn.Linear(512 + 128, 256),
            nn.Tanh(),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(512 + 128, 512),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(512),
            nn.Dropout(0.4),
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(256),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )
        
        # Initialize weights
        self._initialize_weights()
    
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.BatchNorm1d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)
    
    def forward(self, x_img, x_features):
        # Enhanced CNN feature extraction with residual connections
        x_img = self.pool(F.relu(self.bn1(self.conv1(x_img)), inplace=True))
        x_img = self.dropout2d(x_img)
        
        x_img = self.pool(F.relu(self.bn2(self.conv2(x_img)), inplace=True))
        x_img = self.dropout2d(x_img)
        
        x_img = self.pool(F.relu(self.bn3(self.conv3(x_img)), inplace=True))
        x_img = self.dropout2d(x_img)
        
        x_img = self.pool(F.relu(self.bn4(self.conv4(x_img)), inplace=True))
        x_img = self.dropout2d(x_img)
        
        x_img = F.relu(self.bn5(self.conv5(x_img)), inplace=True)
        
        x_img = self.adaptive_pool(x_img)
        x_img = torch.flatten(x_img, 1)
        x_img = self.fc_cnn(x_img)
        
        # Handcrafted features processing
        x_features = self.fc_handcrafted(x_features)
        
        # Combine features with attention
        combined = torch.cat((x_img, x_features), dim=1)
        attention_weight = self.attention(combined)
        combined = combined * attention_weight
        
        x = self.classifier(combined)
        
        return x

# Enhanced Custom Dataset with data augmentation and caching
class PokemonDataset(Dataset):
    def __init__(self, dataframe, transform=None, feature_scaler=None, cache_features=True):
        self.dataframe = dataframe.reset_index(drop=True)
        self.transform = transform
        self.feature_scaler = feature_scaler
        self.cache_features = cache_features
        self.feature_cache = {}
        
        logger.info(f"Initializing dataset with {len(self.dataframe)} samples")
        
        # Extract feature columns
        self.feature_columns = [
            'region_area', 'region_perimeter', 'region_eccentricity', 'region_solidity',
            'region_extent', 'region_circularity', 'region_aspect_ratio', 'region_euler_number',
            'intensity_mean', 'intensity_std', 'intensity_min', 'intensity_max',
            'intensity_median', 'intensity_q25', 'intensity_q75',
            'color_r_mean', 'color_g_mean', 'color_b_mean',
            'color_r_std', 'color_g_std', 'color_b_std',
            'alpha_mean', 'alpha_std', 'alpha_min', 'alpha_max'
        ]
        
        # Target columns
        self.target_columns = [
            'bug', 'dark', 'dragon', 'electric', 'fairy', 'fighting',
            'fire', 'flying', 'ghost', 'grass', 'ground', 'ice',
            'normal', 'poison', 'psychic', 'rock', 'steel', 'water'
        ]
        
        # Pre-process features if scaler is provided
        if self.feature_scaler is not None:
            features_array = self.dataframe[self.feature_columns].values
            features_scaled = self.feature_scaler.transform(features_array)
            for i, col in enumerate(self.feature_columns):
                self.dataframe[col] = features_scaled[:, i]
            logger.info("Applied feature scaling to dataset")
    
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, idx):
        try:
            img_path = self.dataframe.iloc[idx]['image_path']
            
            # Load and transform image
            image = Image.open(img_path).convert('RGB')
            if self.transform:
                image = self.transform(image)
            
            # Get handcrafted features (use cache if available)
            if self.cache_features and idx in self.feature_cache:
                features = self.feature_cache[idx]
            else:
                features = self.dataframe.iloc[idx][self.feature_columns].values.astype(np.float32)
                if self.cache_features:
                    self.feature_cache[idx] = features
            
            # Get targets
            targets = self.dataframe.iloc[idx][self.target_columns].values.astype(np.float32)
            
            return image, torch.tensor(features, dtype=torch.float32), torch.tensor(targets, dtype=torch.float32)
        
        except Exception as e:
            logger.error(f"Error loading sample {idx}: {str(e)}")
            # Return a dummy sample to avoid breaking the training
            dummy_image = torch.zeros(3, 128, 128)
            dummy_features = torch.zeros(len(self.feature_columns), dtype=torch.float32)
            dummy_targets = torch.zeros(len(self.target_columns), dtype=torch.float32)
            return dummy_image, dummy_features, dummy_targets

# Enhanced training function with mixed precision and gradient clipping
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=25, 
                use_mixed_precision=True, gradient_clip_val=1.0):
    
    logger.info("Starting model training...")
    logger.info(f"Training parameters: epochs={num_epochs}, mixed_precision={use_mixed_precision}, gradient_clip={gradient_clip_val}")
    
    best_val_loss = float('inf')
    best_val_acc = 0.0
    patience_counter = 0
    patience = 10
    
    # Mixed precision training
    scaler = torch.cuda.amp.GradScaler() if use_mixed_precision else None
    
    # Training history
    history = {
        'train_loss': [], 'train_acc': [],
        'val_loss': [], 'val_acc': [],
        'lr': []
    }
    
    start_time = time.time()
    
    for epoch in range(num_epochs):
        epoch_start_time = time.time()
        
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        # Training phase
        train_pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training")
        for batch_idx, (images, features, labels) in enumerate(train_pbar):
            try:
                images = images.to(device, non_blocking=True)
                features = features.to(device, non_blocking=True)
                labels = labels.to(device, non_blocking=True)
                
                optimizer.zero_grad()
                
                # Mixed precision forward pass
                if use_mixed_precision:
                    with torch.cuda.amp.autocast():
                        outputs = model(images, features)
                        loss = criterion(outputs, labels)
                    
                    scaler.scale(loss).backward()
                    
                    # Gradient clipping
                    if gradient_clip_val > 0:
                        scaler.unscale_(optimizer)
                        torch.nn.utils.clip_grad_norm_(model.parameters(), gradient_clip_val)
                    
                    scaler.step(optimizer)
                    scaler.update()
                else:
                    outputs = model(images, features)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    
                    # Gradient clipping
                    if gradient_clip_val > 0:
                        torch.nn.utils.clip_grad_norm_(model.parameters(), gradient_clip_val)
                    
                    optimizer.step()
                
                running_loss += loss.item() * images.size(0)
                
                # Calculate accuracy
                with torch.no_grad():
                    preds = torch.sigmoid(outputs) > 0.5
                    correct += (preds == labels.byte()).sum().item()
                    total += labels.numel()
                
                # Update progress bar
                train_pbar.set_postfix({
                    'loss': f'{loss.item():.4f}',
                    'acc': f'{correct/total:.4f}' if total > 0 else '0.0000'
                })
                
                # Clear cache periodically
                if batch_idx % 50 == 0:
                    torch.cuda.empty_cache()
                    
            except Exception as e:
                logger.error(f"Error in training batch {batch_idx}: {str(e)}")
                continue
        
        train_loss = running_loss / len(train_loader.dataset)
        train_acc = correct / total if total > 0 else 0
        
        # Validation phase
        logger.info("Running validation...")
        val_loss, val_acc = evaluate_model(model, val_loader, criterion)
        
        # Update learning rate scheduler
        if isinstance(scheduler, ReduceLROnPlateau):
            scheduler.step(val_loss)
        else:
            scheduler.step()
        
        current_lr = optimizer.param_groups[0]['lr']
        
        # Log epoch results
        epoch_time = time.time() - epoch_start_time
        logger.info(f"Epoch {epoch+1}/{num_epochs} completed in {epoch_time:.2f}s")
        logger.info(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
        logger.info(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")
        logger.info(f"Learning Rate: {current_lr:.6f}")
        
        # Store history
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        history['lr'].append(current_lr)
        
        # Save best model based on validation accuracy
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_val_loss = val_loss
            patience_counter = 0
            
            model_path = "models/pokemon_cnn_best_model.pt"
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'best_val_loss': best_val_loss,
                'best_val_acc': best_val_acc,
                'history': history
            }, model_path)
            logger.success(f"New best model saved! Val Acc: {val_acc:.4f}")
        else:
            patience_counter += 1
            logger.info(f"No improvement. Patience: {patience_counter}/{patience}")
        
        # Early stopping
        if patience_counter >= patience:
            logger.warning(f"Early stopping triggered after {patience} epochs without improvement")
            break
        
        logger.info('-' * 70)
        
        # Memory cleanup
        gc.collect()
        torch.cuda.empty_cache()
    
    total_time = time.time() - start_time
    logger.success(f"Training completed in {total_time:.2f}s")
    logger.info(f"Best validation accuracy: {best_val_acc:.4f}")
    
    return model, history

# Enhanced evaluation function
def evaluate_model(model, data_loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, features, labels in data_loader:
            try:
                images = images.to(device, non_blocking=True)
                features = features.to(device, non_blocking=True)
                labels = labels.to(device, non_blocking=True)
                
                outputs = model(images, features)
                loss = criterion(outputs, labels)
                
                running_loss += loss.item() * images.size(0)
                
                preds = torch.sigmoid(outputs) > 0.5
                correct += (preds == labels.byte()).sum().item()
                total += labels.numel()
                
            except Exception as e:
                logger.error(f"Error in evaluation batch: {str(e)}")
                continue
    
    loss = running_loss / len(data_loader.dataset) if len(data_loader.dataset) > 0 else float('inf')
    acc = correct / total if total > 0 else 0
    
    return loss, acc

# Feature preprocessing function
def preprocess_features(train_df, val_df, feature_columns):
    """Standardize handcrafted features"""
    logger.info("Preprocessing handcrafted features...")
    
    scaler = StandardScaler()
    train_features = train_df[feature_columns].values
    scaler.fit(train_features)
    
    # Transform features
    train_features_scaled = scaler.transform(train_features)
    val_features_scaled = scaler.transform(val_df[feature_columns].values)
    
    # Update dataframes
    for i, col in enumerate(feature_columns):
        train_df[col] = train_features_scaled[:, i]
        val_df[col] = val_features_scaled[:, i]
    
    logger.info("Feature preprocessing completed")
    return train_df, val_df, scaler

# Enhanced training setup and execution
def run_training(df_pokemon_features_final2):
    logger.info("="*70)
    logger.info("STARTING POKEMON CNN TRAINING")
    logger.info("="*70)
    
    # Check device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    logger.info(f"Using device: {device}")
    if torch.cuda.is_available():
        logger.info(f"GPU: {torch.cuda.get_device_name()}")
        logger.info(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    
    # Split data
    logger.info("Splitting dataset...")
    train_df, val_df = train_test_split(df_pokemon_features_final2, test_size=0.2, 
                                       random_state=42, stratify=None)
    logger.info(f"Train samples: {len(train_df)}, Validation samples: {len(val_df)}")
    
    # Enhanced transforms with data augmentation
    train_transform = transforms.Compose([
        transforms.Resize((144, 144)),  # Slightly larger for random crop
        transforms.RandomCrop((128, 128)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomRotation(degrees=15),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    val_transform = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    # Preprocess features
    feature_columns = [
        'region_area', 'region_perimeter', 'region_eccentricity', 'region_solidity',
        'region_extent', 'region_circularity', 'region_aspect_ratio', 'region_euler_number',
        'intensity_mean', 'intensity_std', 'intensity_min', 'intensity_max',
        'intensity_median', 'intensity_q25', 'intensity_q75',
        'color_r_mean', 'color_g_mean', 'color_b_mean',
        'color_r_std', 'color_g_std', 'color_b_std',
        'alpha_mean', 'alpha_std', 'alpha_min', 'alpha_max'
    ]
    
    train_df_processed, val_df_processed, feature_scaler = preprocess_features(
        train_df.copy(), val_df.copy(), feature_columns
    )
    
    # Create datasets
    logger.info("Creating datasets...")
    train_dataset = PokemonDataset(train_df_processed, transform=train_transform, 
                                 feature_scaler=None, cache_features=True)
    val_dataset = PokemonDataset(val_df_processed, transform=val_transform, 
                               feature_scaler=None, cache_features=True)
    
    # Create dataloaders with optimized settings
    batch_size = 32
    num_workers = min(4, os.cpu_count())
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, 
                            num_workers=num_workers, pin_memory=True, 
                            persistent_workers=True, prefetch_factor=2)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, 
                          num_workers=num_workers, pin_memory=True, 
                          persistent_workers=True, prefetch_factor=2)
    
    logger.info(f"DataLoader settings: batch_size={batch_size}, num_workers={num_workers}")
    
    # Initialize enhanced model
    logger.info("Initializing model...")
    model = PokemonCNN(num_classes=18, num_handcrafted_features=26).to(device)
    
    # Count parameters
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    logger.info(f"Model parameters: {total_params:,} total, {trainable_params:,} trainable")
    
    # Enhanced optimizer with fixed learning rate
    criterion = nn.BCEWithLogitsLoss(pos_weight=None)  # Can add class weights if needed
    optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)
    
    # Fixed learning rate scheduler - no learning rate changes
    scheduler = None  # No scheduler for constant learning rate
    
    logger.info("Training configuration:")
    logger.info(f"Optimizer: {type(optimizer).__name__}")
    logger.info(f"Scheduler: None (fixed learning rate)")
    logger.info(f"Criterion: {type(criterion).__name__}")
    
    # Train the model
    model, history = train_model(model, train_loader, val_loader, criterion, optimizer, 
                               scheduler, num_epochs=50, use_mixed_precision=True, 
                               gradient_clip_val=1.0)
    
    logger.success("Training completed successfully!")
    return model, history, feature_scaler

# Set global device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ADDITIONAL UTILITY FUNCTIONS FOR MONITORING AND ANALYSIS

# Complete the plot_training_history function
def plot_training_history(history):
    """Plot training history for analysis"""
    import matplotlib.pyplot as plt
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Loss plot
    axes[0, 0].plot(history['train_loss'], label='Train Loss', color='blue')
    axes[0, 0].plot(history['val_loss'], label='Val Loss', color='red')
    axes[0, 0].set_title('Training and Validation Loss')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].legend()
    axes[0, 0].grid(True)
    
    # Accuracy plot
    axes[0, 1].plot(history['train_acc'], label='Train Acc', color='blue')
    axes[0, 1].plot(history['val_acc'], label='Val Acc', color='red')
    axes[0, 1].set_title('Training and Validation Accuracy')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Accuracy')
    axes[0, 1].legend()
    axes[0, 1].grid(True)
    
    # Learning rate plot
    axes[1, 0].plot(history['lr'], label='Learning Rate', color='green')
    axes[1, 0].set_title('Learning Rate Schedule')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Learning Rate')
    axes[1, 0].set_yscale('log')
    axes[1, 0].legend()
    axes[1, 0].grid(True)
    
    # Loss difference plot
    loss_diff = [abs(t - v) for t, v in zip(history['train_loss'], history['val_loss'])]
    axes[1, 1].plot(loss_diff, label='Train-Val Loss Diff', color='purple')
    axes[1, 1].set_title('Training-Validation Loss Difference')
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('Loss Difference')
    axes[1, 1].legend()
    axes[1, 1].grid(True)
    
    plt.tight_layout()
    plt.savefig('logs/training_history.png', dpi=300, bbox_inches='tight')
    plt.show()
    logger.info("Training history plots saved to logs/training_history.png")

def save_model_info(model, feature_scaler, history, save_path="models/"):
    """Save model information and scaler"""
    import pickle
    
    # Save the feature scaler
    scaler_path = os.path.join(save_path, "feature_scaler.pkl")
    with open(scaler_path, 'wb') as f:
        pickle.dump(feature_scaler, f)
    logger.info(f"Feature scaler saved to {scaler_path}")
    
    # Save training history
    history_path = os.path.join(save_path, "training_history.pkl")
    with open(history_path, 'wb') as f:
        pickle.dump(history, f)
    logger.info(f"Training history saved to {history_path}")
    
    # Save model summary
    summary_path = os.path.join(save_path, "model_summary.txt")
    with open(summary_path, 'w') as f:
        f.write("Pokemon CNN Model Summary\n")
        f.write("=" * 50 + "\n")
        f.write(f"Total parameters: {sum(p.numel() for p in model.parameters()):,}\n")
        f.write(f"Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}\n")
        f.write(f"Best validation accuracy: {max(history['val_acc']):.4f}\n")
        f.write(f"Best validation loss: {min(history['val_loss']):.4f}\n")
        f.write(f"Total epochs trained: {len(history['train_loss'])}\n")
    logger.info(f"Model summary saved to {summary_path}")

def load_trained_model(model_path="models/pokemon_cnn_best_model.pt", num_classes=18, num_handcrafted_features=26):
    """Load a trained model for inference"""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Initialize model
    model = PokemonCNN(num_classes=num_classes, num_handcrafted_features=num_handcrafted_features)
    
    # Load checkpoint
    checkpoint = torch.load(model_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.to(device)
    model.eval()
    
    logger.info(f"Model loaded from {model_path}")
    logger.info(f"Model was trained for {checkpoint['epoch']} epochs")
    logger.info(f"Best validation accuracy: {checkpoint['best_val_acc']:.4f}")
    
    return model, checkpoint

def predict_pokemon_types(model, image_path, feature_scaler, type_labels=None):
    """Make predictions on a single Pokemon image"""
    if type_labels is None:
        type_labels = ['bug', 'dark', 'dragon', 'electric', 'fairy', 'fighting',
                      'fire', 'flying', 'ghost', 'grass', 'ground', 'ice',
                      'normal', 'poison', 'psychic', 'rock', 'steel', 'water']
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Transform for inference
    transform = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    # Load and preprocess image
    image = Image.open(image_path).convert('RGB')
    image_tensor = transform(image).unsqueeze(0).to(device)
    
    # Extract handcrafted features
    features_dict = extract_image_features(image_path)
    features_array = np.array([features_dict[key] for key in sorted(features_dict.keys())]).reshape(1, -1)
    features_scaled = feature_scaler.transform(features_array)
    features_tensor = torch.tensor(features_scaled, dtype=torch.float32).to(device)
    
    # Make prediction
    with torch.no_grad():
        outputs = model(image_tensor, features_tensor)
        probabilities = torch.sigmoid(outputs).cpu().numpy()[0]
    
    # Create results
    results = []
    for i, (type_name, prob) in enumerate(zip(type_labels, probabilities)):
        results.append({
            'type': type_name,
            'probability': float(prob),
            'predicted': bool(prob > 0.5)
        })
    
    # Sort by probability
    results.sort(key=lambda x: x['probability'], reverse=True)
    
    return results

# Add missing imports at the top of your file (if not already present)
"""
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torchvision.transforms as transforms

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import skimage
import skimage.io
import skimage.color
import skimage.measure

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

import os
import time
import gc
from tqdm import tqdm
from loguru import logger
import pickle
"""

'\nimport torch\nimport torch.nn as nn\nimport torch.nn.functional as F\nimport torch.optim as optim\nfrom torch.utils.data import Dataset, DataLoader\nfrom torch.optim.lr_scheduler import ReduceLROnPlateau\nimport torchvision.transforms as transforms\n\nimport numpy as np\nimport pandas as pd\nimport matplotlib.pyplot as plt\nimport seaborn as sns\nfrom PIL import Image\nimport skimage\nimport skimage.io\nimport skimage.color\nimport skimage.measure\n\nfrom sklearn.model_selection import train_test_split\nfrom sklearn.preprocessing import StandardScaler\nfrom sklearn.metrics import classification_report, confusion_matrix\n\nimport os\nimport time\nimport gc\nfrom tqdm import tqdm\nfrom loguru import logger\nimport pickle\n'

In [ ]:
# Main execution (add this at the end of your script)
if __name__ == "__main__":
    # Assuming you have df_pokemon_features_final2 loaded
    # model, history, feature_scaler = run_training(df_pokemon_features_final2)
    
    # After training, you can:
    # plot_training_history(history)
    # save_model_info(model, feature_scaler, history)
    
    # For inference later:
    # model, checkpoint = load_trained_model()
    # results = predict_pokemon_types(model, "path/to/pokemon/image.png", feature_scaler)
    # print("Predicted types:", [r for r in results if r['predicted']])
    
    pass

In [ ]:
model, history, feature_scaler = run_training(df_pokemon_features_final2)

2025-06-25 19:49:19.485 | INFO     | __main__:run_training:503 - ======================================================================
2025-06-25 19:49:19.486 | INFO     | __main__:run_training:504 - STARTING POKEMON CNN TRAINING
2025-06-25 19:49:19.488 | INFO     | __main__:run_training:505 - ======================================================================
2025-06-25 19:49:19.489 | INFO     | __main__:run_training:509 - Using device: cuda
2025-06-25 19:49:19.493 | INFO     | __main__:run_training:511 - GPU: NVIDIA GeForce RTX 3070 Laptop GPU
2025-06-25 19:49:19.494 | INFO     | __main__:run_training:512 - GPU Memory: 8.6 GB
2025-06-25 19:49:19.494 | INFO     | __main__:run_training:515 - Splitting dataset...
2025-06-25 19:49:19.498 | INFO     | __main__:run_training:518 - Train samples: 699, Validation samples: 175
2025-06-25 19:49:19.500 | INFO     | __main__:preprocess_features:483 - Preprocessing handcrafted features...
2025-06-25 19:49:19.512 | INFO     | __main__:preproces

19:49:19 | INFO | ======================================================================
19:49:19 | INFO | STARTING POKEMON CNN TRAINING
19:49:19 | INFO | ======================================================================
19:49:19 | INFO | Using device: cuda
19:49:19 | INFO | GPU: NVIDIA GeForce RTX 3070 Laptop GPU
19:49:19 | INFO | GPU Memory: 8.6 GB
19:49:19 | INFO | Splitting dataset...
19:49:19 | INFO | Train samples: 699, Validation samples: 175
19:49:19 | INFO | Preprocessing handcrafted features...
19:49:19 | INFO | Feature preprocessing completed
19:49:19 | INFO | Creating datasets...
19:49:19 | INFO | Initializing dataset with 699 samples
19:49:19 | INFO | Initializing dataset with 175 samples
19:49:19 | INFO | DataLoader settings: batch_size=32, num_workers=4
19:49:19 | INFO | Initializing model...
19:49:19 | INFO | Initializing PokemonCNN with 18 classes and 26 handcrafted features


2025-06-25 19:49:19.889 | INFO     | __main__:run_training:579 - Model parameters: 13,501,843 total, 13,501,843 trainable
2025-06-25 19:49:19.891 | INFO     | __main__:run_training:588 - Training configuration:
2025-06-25 19:49:19.892 | INFO     | __main__:run_training:589 - Optimizer: AdamW
2025-06-25 19:49:19.892 | INFO     | __main__:run_training:590 - Scheduler: None (fixed learning rate)
2025-06-25 19:49:19.893 | INFO     | __main__:run_training:591 - Criterion: BCEWithLogitsLoss
2025-06-25 19:49:19.894 | INFO     | __main__:train_model:295 - Starting model training...
2025-06-25 19:49:19.895 | INFO     | __main__:train_model:296 - Training parameters: epochs=50, mixed_precision=True, gradient_clip=1.0
C:\Users\Admin\AppData\Local\Temp\ipykernel_44896\3359893606.py:304: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler() if use_mixed_precision else None


19:49:19 | INFO | Model parameters: 13,501,843 total, 13,501,843 trainable
19:49:19 | INFO | Training configuration:
19:49:19 | INFO | Optimizer: AdamW
19:49:19 | INFO | Scheduler: None (fixed learning rate)
19:49:19 | INFO | Criterion: BCEWithLogitsLoss
19:49:19 | INFO | Starting model training...
19:49:19 | INFO | Training parameters: epochs=50, mixed_precision=True, gradient_clip=1.0


Epoch 1/50 - Training:   0%|          | 0/22 [00:00<?, ?it/s]

In [ ]:
plot_training_history(history)

In [ ]:
save_model_info(model, feature_scaler, history)

In [ ]:
model, checkpoint = load_trained_model()

In [ ]:
results = predict_pokemon_types(model, "test_set/pikachu_25.png", feature_scaler)

In [ ]:
 print("Predicted types:", [r for r in results if r['predicted']])

In [19]:
# Configure loguru logger
logger.add("logs/pokemon_training_{time}.log", 
           format="{time:YYYY-MM-DD HH:mm:ss} | {level} | {message}",
           level="INFO")
logger.add(lambda msg: print(msg, end=""), 
           format="{time:HH:mm:ss} | {level} | {message}",
           level="INFO", colorize=True)

# Create directories if they don't exist
os.makedirs("models", exist_ok=True)
os.makedirs("logs", exist_ok=True)

# Keep your original function exactly as is
def extract_image_features(image_path):
    image = skimage.io.imread(image_path)

    # Handle RGB or RGBA
    if image.shape[-1] == 4:
        rgb = image[..., :3]
        alpha = image[..., 3] / 255.0
    else:
        rgb = image
        alpha = np.ones(rgb.shape[:2])

    # Grayscale from RGB
    rgb_gray = skimage.color.rgb2gray(rgb)

    # Binary mask for region detection (ignore transparency)
    thresh = rgb_gray > 0.1
    labeled = skimage.measure.label(thresh)
    regions = skimage.measure.regionprops(labeled, intensity_image=rgb_gray)

    if len(regions) == 0:
        return {  # Return zeros with names
            'region_area': 0, 'region_perimeter': 0, 'region_eccentricity': 0,
            'region_solidity': 0, 'region_extent': 0, 'region_circularity': 0,
            'region_aspect_ratio': 0, 'region_euler_number': 0,
            'intensity_mean': 0, 'intensity_std': 0, 'intensity_min': 0, 'intensity_max': 0,
            'intensity_median': 0, 'intensity_q25': 0, 'intensity_q75': 0,
            'color_r_mean': 0, 'color_g_mean': 0, 'color_b_mean': 0,
            'color_r_std': 0, 'color_g_std': 0, 'color_b_std': 0,
            'alpha_mean': 0, 'alpha_std': 0, 'alpha_min': 0, 'alpha_max': 0
        }

    region = max(regions, key=lambda r: r.area)

    # Shape features
    shape_features = {
        'region_area': region.area,
        'region_perimeter': region.perimeter,
        'region_eccentricity': region.eccentricity,
        'region_solidity': region.solidity,
        'region_extent': region.extent,
        'region_circularity': region.perimeter / (4 * np.sqrt(region.area)) if region.area > 0 else 0,
        'region_aspect_ratio': region.major_axis_length / (region.minor_axis_length + 1e-10),
        'region_euler_number': region.euler_number
    }

    # Intensity
    coords = region.coords
    intensities = rgb_gray[coords[:, 0], coords[:, 1]]
    intensity_features = {
        'intensity_mean': intensities.mean(),
        'intensity_std': intensities.std() if len(intensities) > 1 else 0,
        'intensity_min': np.min(intensities),
        'intensity_max': np.max(intensities),
        'intensity_median': np.median(intensities),
        'intensity_q25': np.percentile(intensities, 25),
        'intensity_q75': np.percentile(intensities, 75)
    }

    # Colors
    region_mask = labeled == region.label
    region_pixels = rgb[region_mask]
    color_means = region_pixels.mean(axis=0) if region_pixels.size else [0, 0, 0]
    color_stds = region_pixels.std(axis=0) if region_pixels.size else [0, 0, 0]
    color_features = {
        'color_r_mean': color_means[0],
        'color_g_mean': color_means[1],
        'color_b_mean': color_means[2],
        'color_r_std': color_stds[0],
        'color_g_std': color_stds[1],
        'color_b_std': color_stds[2],
    }

    # Alpha
    alpha_pixels = alpha[region_mask]
    alpha_features = {
        'alpha_mean': alpha_pixels.mean() if alpha_pixels.size else 0,
        'alpha_std': alpha_pixels.std() if alpha_pixels.size else 0,
        'alpha_min': alpha_pixels.min() if alpha_pixels.size else 0,
        'alpha_max': alpha_pixels.max() if alpha_pixels.size else 0,
    }

    # Combine all
    features = {}
    features.update(shape_features)
    features.update(intensity_features)
    features.update(color_features)
    features.update(alpha_features)

    return features

# Optimized CNN model with improved architecture
class PokemonCNN(nn.Module):
    def __init__(self, num_classes=18, num_handcrafted_features=26):
        super(PokemonCNN, self).__init__()
        
        logger.info(f"Initializing PokemonCNN with {num_classes} classes and {num_handcrafted_features} handcrafted features")
        
        # Enhanced CNN feature extractor with residual connections
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(512)
        self.conv5 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.bn5 = nn.BatchNorm2d(512)
        
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout2d = nn.Dropout2d(0.2)
        
        # Adaptive pooling to handle different input sizes
        self.adaptive_pool = nn.AdaptiveAvgPool2d((4, 4))
        
        # Enhanced fully connected layers for CNN features
        self.fc_cnn = nn.Sequential(
            nn.Linear(512 * 4 * 4, 1024),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(1024),
            nn.Dropout(0.5),
            nn.Linear(1024, 512),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(512),
            nn.Dropout(0.3)
        )
        
        # Enhanced fully connected layers for handcrafted features
        self.fc_handcrafted = nn.Sequential(
            nn.Linear(num_handcrafted_features, 256),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(256),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(128),
            nn.Dropout(0.2)
        )
        
        # Enhanced combined classifier with attention mechanism
        self.attention = nn.Sequential(
            nn.Linear(512 + 128, 256),
            nn.Tanh(),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(512 + 128, 512),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(512),
            nn.Dropout(0.4),
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(256),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )
        
        # Initialize weights
        self._initialize_weights()
    
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.BatchNorm1d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)
    
    def forward(self, x_img, x_features):
        # Enhanced CNN feature extraction with residual connections
        x_img = self.pool(F.relu(self.bn1(self.conv1(x_img)), inplace=True))
        x_img = self.dropout2d(x_img)
        
        x_img = self.pool(F.relu(self.bn2(self.conv2(x_img)), inplace=True))
        x_img = self.dropout2d(x_img)
        
        x_img = self.pool(F.relu(self.bn3(self.conv3(x_img)), inplace=True))
        x_img = self.dropout2d(x_img)
        
        x_img = self.pool(F.relu(self.bn4(self.conv4(x_img)), inplace=True))
        x_img = self.dropout2d(x_img)
        
        x_img = F.relu(self.bn5(self.conv5(x_img)), inplace=True)
        
        x_img = self.adaptive_pool(x_img)
        x_img = torch.flatten(x_img, 1)
        x_img = self.fc_cnn(x_img)
        
        # Handcrafted features processing
        x_features = self.fc_handcrafted(x_features)
        
        # Combine features with attention
        combined = torch.cat((x_img, x_features), dim=1)
        attention_weight = self.attention(combined)
        combined = combined * attention_weight
        
        x = self.classifier(combined)
        
        return x

# Enhanced Custom Dataset with data augmentation and caching
class PokemonDataset(Dataset):
    def __init__(self, dataframe, transform=None, feature_scaler=None, cache_features=True):
        self.dataframe = dataframe.reset_index(drop=True)
        self.transform = transform
        self.feature_scaler = feature_scaler
        self.cache_features = cache_features
        self.feature_cache = {}
        
        logger.info(f"Initializing dataset with {len(self.dataframe)} samples")
        
        # Extract feature columns
        self.feature_columns = [
            'region_area', 'region_perimeter', 'region_eccentricity', 'region_solidity',
            'region_extent', 'region_circularity', 'region_aspect_ratio', 'region_euler_number',
            'intensity_mean', 'intensity_std', 'intensity_min', 'intensity_max',
            'intensity_median', 'intensity_q25', 'intensity_q75',
            'color_r_mean', 'color_g_mean', 'color_b_mean',
            'color_r_std', 'color_g_std', 'color_b_std',
            'alpha_mean', 'alpha_std', 'alpha_min', 'alpha_max'
        ]
        
        # Target columns
        self.target_columns = [
            'bug', 'dark', 'dragon', 'electric', 'fairy', 'fighting',
            'fire', 'flying', 'ghost', 'grass', 'ground', 'ice',
            'normal', 'poison', 'psychic', 'rock', 'steel', 'water'
        ]
        
        # Pre-process features if scaler is provided
        if self.feature_scaler is not None:
            features_array = self.dataframe[self.feature_columns].values
            features_scaled = self.feature_scaler.transform(features_array)
            for i, col in enumerate(self.feature_columns):
                self.dataframe[col] = features_scaled[:, i]
            logger.info("Applied feature scaling to dataset")
    
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, idx):
        try:
            img_path = self.dataframe.iloc[idx]['image_path']
            
            # Load and transform image
            image = Image.open(img_path).convert('RGB')
            if self.transform:
                image = self.transform(image)
            
            # Get handcrafted features (use cache if available)
            if self.cache_features and idx in self.feature_cache:
                features = self.feature_cache[idx]
            else:
                features = self.dataframe.iloc[idx][self.feature_columns].values.astype(np.float32)
                if self.cache_features:
                    self.feature_cache[idx] = features
            
            # Get targets
            targets = self.dataframe.iloc[idx][self.target_columns].values.astype(np.float32)
            
            return image, torch.tensor(features, dtype=torch.float32), torch.tensor(targets, dtype=torch.float32)
        
        except Exception as e:
            logger.error(f"Error loading sample {idx}: {str(e)}")
            # Return a dummy sample to avoid breaking the training
            dummy_image = torch.zeros(3, 128, 128)
            dummy_features = torch.zeros(len(self.feature_columns), dtype=torch.float32)
            dummy_targets = torch.zeros(len(self.target_columns), dtype=torch.float32)
            return dummy_image, dummy_features, dummy_targets

# Enhanced training function with mixed precision and gradient clipping
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=25, 
                use_mixed_precision=True, gradient_clip_val=1.0):
    
    logger.info("Starting model training...")
    logger.info(f"Training parameters: epochs={num_epochs}, mixed_precision={use_mixed_precision}, gradient_clip={gradient_clip_val}")
    
    best_val_loss = float('inf')
    best_val_acc = 0.0
    patience_counter = 0
    patience = 10
    
    # Mixed precision training
    scaler = torch.cuda.amp.GradScaler() if use_mixed_precision else None
    
    # Training history
    history = {
        'train_loss': [], 'train_acc': [],
        'val_loss': [], 'val_acc': [],
        'lr': []
    }
    
    start_time = time.time()
    
    for epoch in range(num_epochs):
        epoch_start_time = time.time()
        
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        # Training phase
        train_pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training")
        for batch_idx, (images, features, labels) in enumerate(train_pbar):
            try:
                images = images.to(device, non_blocking=True)
                features = features.to(device, non_blocking=True)
                labels = labels.to(device, non_blocking=True)
                
                optimizer.zero_grad()
                
                # Mixed precision forward pass
                if use_mixed_precision:
                    with torch.cuda.amp.autocast():
                        outputs = model(images, features)
                        loss = criterion(outputs, labels)
                    
                    scaler.scale(loss).backward()
                    
                    # Gradient clipping
                    if gradient_clip_val > 0:
                        scaler.unscale_(optimizer)
                        torch.nn.utils.clip_grad_norm_(model.parameters(), gradient_clip_val)
                    
                    scaler.step(optimizer)
                    scaler.update()
                else:
                    outputs = model(images, features)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    
                    # Gradient clipping
                    if gradient_clip_val > 0:
                        torch.nn.utils.clip_grad_norm_(model.parameters(), gradient_clip_val)
                    
                    optimizer.step()
                
                running_loss += loss.item() * images.size(0)
                
                # Calculate accuracy
                with torch.no_grad():
                    preds = torch.sigmoid(outputs) > 0.5
                    correct += (preds == labels.byte()).sum().item()
                    total += labels.numel()
                
                # Update progress bar
                train_pbar.set_postfix({
                    'loss': f'{loss.item():.4f}',
                    'acc': f'{correct/total:.4f}' if total > 0 else '0.0000'
                })
                
                # Clear cache periodically
                if batch_idx % 50 == 0:
                    torch.cuda.empty_cache()
                    
            except Exception as e:
                logger.error(f"Error in training batch {batch_idx}: {str(e)}")
                continue
        
        train_loss = running_loss / len(train_loader.dataset)
        train_acc = correct / total if total > 0 else 0
        
        # Validation phase
        logger.info("Running validation...")
        val_loss, val_acc = evaluate_model(model, val_loader, criterion)
        
        # Update learning rate scheduler
        if isinstance(scheduler, ReduceLROnPlateau):
            scheduler.step(val_loss)
        else:
            scheduler.step()
        
        current_lr = optimizer.param_groups[0]['lr']
        
        # Log epoch results
        epoch_time = time.time() - epoch_start_time
        logger.info(f"Epoch {epoch+1}/{num_epochs} completed in {epoch_time:.2f}s")
        logger.info(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
        logger.info(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")
        logger.info(f"Learning Rate: {current_lr:.6f}")
        
        # Store history
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        history['lr'].append(current_lr)
        
        # Save best model based on validation accuracy
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_val_loss = val_loss
            patience_counter = 0
            
            model_path = "models/pokemon_cnn_best_model.pt"
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'best_val_loss': best_val_loss,
                'best_val_acc': best_val_acc,
                'history': history
            }, model_path)
            logger.success(f"New best model saved! Val Acc: {val_acc:.4f}")
        else:
            patience_counter += 1
            logger.info(f"No improvement. Patience: {patience_counter}/{patience}")
        
        # Early stopping
        if patience_counter >= patience:
            logger.warning(f"Early stopping triggered after {patience} epochs without improvement")
            break
        
        logger.info('-' * 70)
        
        # Memory cleanup
        gc.collect()
        torch.cuda.empty_cache()
    
    total_time = time.time() - start_time
    logger.success(f"Training completed in {total_time:.2f}s")
    logger.info(f"Best validation accuracy: {best_val_acc:.4f}")
    
    return model, history

# Enhanced evaluation function
def evaluate_model(model, data_loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, features, labels in data_loader:
            try:
                images = images.to(device, non_blocking=True)
                features = features.to(device, non_blocking=True)
                labels = labels.to(device, non_blocking=True)
                
                outputs = model(images, features)
                loss = criterion(outputs, labels)
                
                running_loss += loss.item() * images.size(0)
                
                preds = torch.sigmoid(outputs) > 0.5
                correct += (preds == labels.byte()).sum().item()
                total += labels.numel()
                
            except Exception as e:
                logger.error(f"Error in evaluation batch: {str(e)}")
                continue
    
    loss = running_loss / len(data_loader.dataset) if len(data_loader.dataset) > 0 else float('inf')
    acc = correct / total if total > 0 else 0
    
    return loss, acc

# Feature preprocessing function
def preprocess_features(train_df, val_df, feature_columns):
    """Standardize handcrafted features"""
    logger.info("Preprocessing handcrafted features...")
    
    scaler = StandardScaler()
    train_features = train_df[feature_columns].values
    scaler.fit(train_features)
    
    # Transform features
    train_features_scaled = scaler.transform(train_features)
    val_features_scaled = scaler.transform(val_df[feature_columns].values)
    
    # Update dataframes
    for i, col in enumerate(feature_columns):
        train_df[col] = train_features_scaled[:, i]
        val_df[col] = val_features_scaled[:, i]
    
    logger.info("Feature preprocessing completed")
    return train_df, val_df, scaler

# Enhanced training setup and execution
def run_training(df_pokemon_features_final2):
    logger.info("="*70)
    logger.info("STARTING POKEMON CNN TRAINING")
    logger.info("="*70)
    
    # Check device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    logger.info(f"Using device: {device}")
    if torch.cuda.is_available():
        logger.info(f"GPU: {torch.cuda.get_device_name()}")
        logger.info(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    
    # Split data
    logger.info("Splitting dataset...")
    train_df, val_df = train_test_split(df_pokemon_features_final2, test_size=0.2, 
                                       random_state=42, stratify=None)
    logger.info(f"Train samples: {len(train_df)}, Validation samples: {len(val_df)}")
    
    # Enhanced transforms with data augmentation
    train_transform = transforms.Compose([
        transforms.Resize((144, 144)),  # Slightly larger for random crop
        transforms.RandomCrop((128, 128)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomRotation(degrees=15),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    val_transform = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    # Preprocess features
    feature_columns = [
        'region_area', 'region_perimeter', 'region_eccentricity', 'region_solidity',
        'region_extent', 'region_circularity', 'region_aspect_ratio', 'region_euler_number',
        'intensity_mean', 'intensity_std', 'intensity_min', 'intensity_max',
        'intensity_median', 'intensity_q25', 'intensity_q75',
        'color_r_mean', 'color_g_mean', 'color_b_mean',
        'color_r_std', 'color_g_std', 'color_b_std',
        'alpha_mean', 'alpha_std', 'alpha_min', 'alpha_max'
    ]
    
    train_df_processed, val_df_processed, feature_scaler = preprocess_features(
        train_df.copy(), val_df.copy(), feature_columns
    )
    
    # Create datasets
    logger.info("Creating datasets...")
    train_dataset = PokemonDataset(train_df_processed, transform=train_transform, 
                                 feature_scaler=None, cache_features=True)
    val_dataset = PokemonDataset(val_df_processed, transform=val_transform, 
                               feature_scaler=None, cache_features=True)
    
    # Create dataloaders with optimized settings
    batch_size = 32
    num_workers = min(4, os.cpu_count())
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, 
                            num_workers=num_workers, pin_memory=True, 
                            persistent_workers=True, prefetch_factor=2)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, 
                          num_workers=num_workers, pin_memory=True, 
                          persistent_workers=True, prefetch_factor=2)
    
    logger.info(f"DataLoader settings: batch_size={batch_size}, num_workers={num_workers}")
    
    # Initialize enhanced model
    logger.info("Initializing model...")
    model = PokemonCNN(num_classes=18, num_handcrafted_features=26).to(device)
    
    # Count parameters
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    logger.info(f"Model parameters: {total_params:,} total, {trainable_params:,} trainable")
    
    # Enhanced optimizer and scheduler
    criterion = nn.BCEWithLogitsLoss(pos_weight=None)  # Can add class weights if needed
    optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)
    
    # Learning rate scheduler
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, 
                                verbose=True, min_lr=1e-6)
    
    logger.info("Training configuration:")
    logger.info(f"Optimizer: {type(optimizer).__name__}")
    logger.info(f"Scheduler: {type(scheduler).__name__}")
    logger.info(f"Criterion: {type(criterion).__name__}")
    
    # Train the model
    model, history = train_model(model, train_loader, val_loader, criterion, optimizer, 
                               scheduler, num_epochs=50, use_mixed_precision=True, 
                               gradient_clip_val=1.0)
    
    logger.success("Training completed successfully!")
    return model, history, feature_scaler

# Set global device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Example usage (uncomment to run):
# model, history, scaler = run_training(df_pokemon_features_final2)

# ADDITIONAL UTILITY FUNCTIONS FOR MONITORING AND ANALYSIS

def plot_training_history(history):
    """Plot training history for analysis"""
    import matplotlib.pyplot as plt
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Loss plot
    axes[0, 0].plot(history['train_loss'], label='Train Loss', color='blue')
    axes[0, 0].plot(history['val_loss'], label='Val Loss', color='red')
    axes[0, 0].set_title('Training and Validation Loss')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].legend()
    axes[0, 0].grid(True)
    
    # Accuracy plot
    axes[0, 1].plot(history['train_acc'], label='Train Acc', color='blue')
    axes[0, 1].plot(history['val_acc'], label='Val Acc', color='red')
    axes[0, 1].set_title('Training and Validation Accuracy')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Accuracy')
    axes[0, 1].legend()
    axes[0, 1].grid(True)
    
    # Learning rate plot
    axes[1, 0].plot(history['lr'], label='Learning Rate', color='green')
    axes[1, 0].set_title('Learning Rate Schedule')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Learning Rate')
    axes[1, 0].set_yscale('log')
    axes[1, 0].legend()
    axes[1, 0].grid(True)
    
    # Loss difference plot
    loss_diff = [abs(t - v) for t, v in zip(history['train_loss'], history['val_loss'])]
    axes[1, 1].plot(loss_diff, label='Train-Val Loss Diff', color='purple')
    axes[1, 1].set_title('Training-Validation Loss Difference')
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('Loss Difference')
    axes[1, 1].legend()
    axes[1, 1].grid(True)
    
    plt.tight_layout()
    plt.savefig('logs/training_history.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    logger.info("Training history plots saved to logs/training_history.png")

def load_best_model(model_path="models/pokemon_cnn_best_model.pt", device=None):
    """Load the best saved model"""
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    logger.info(f"Loading model from {model_path}")
    checkpoint = torch.load(model_path, map_location=device)
    
    model = PokemonCNN(num_classes=18, num_handcrafted_features=26).to(device)
    model.load_state_dict(checkpoint['model_state_dict'])
    
    logger.info(f"Model loaded successfully from epoch {checkpoint['epoch']}")
    logger.info(f"Best validation accuracy: {checkpoint['best_val_acc']:.4f}")
    
    return model, checkpoint

def model_inference(model, image_path, handcrafted_features, feature_scaler, device=None):
    """Run inference on a single image"""
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Prepare image
    transform = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    image = Image.open(image_path).convert('RGB')
    image_tensor = transform(image).unsqueeze(0).to(device)
    
    # Prepare features
    features_scaled = feature_scaler.transform([handcrafted_features])
    features_tensor = torch.tensor(features_scaled, dtype=torch.float32).to(device)
    
    # Inference
    model.eval()
    with torch.no_grad():
        outputs = model(image_tensor, features_tensor)
        probabilities = torch.sigmoid(outputs).cpu().numpy()[0]
    
    # Type names
    type_names = [
        'bug', 'dark', 'dragon', 'electric', 'fairy', 'fighting',
        'fire', 'flying', 'ghost', 'grass', 'ground', 'ice',
        'normal', 'poison', 'psychic', 'rock', 'steel', 'water'
    ]
    
    # Get predictions
    predictions = []
    for i, prob in enumerate(probabilities):
        if prob > 0.5:
            predictions.append((type_names[i], prob))
    
    predictions.sort(key=lambda x: x[1], reverse=True)
    
    logger.info(f"Predictions for {image_path}:")
    for type_name, prob in predictions:
        logger.info(f"  {type_name}: {prob:.4f}")
    
    return predictions

# Example usage after training:
# model, history, scaler = execute_training(df_pokemon_features_final2)
# plot_training_history(history)
# model, checkpoint = load_best_model()
# predictions = model_inference(model, "path/to/pokemon.jpg", handcrafted_features, scaler)

In [18]:
model, history, scaler = run_training(df_pokemon_features_final2)

18:26:35 | INFO | ======================================================================
18:26:35 | INFO | STARTING POKEMON CNN TRAINING
18:26:35 | INFO | ======================================================================
18:26:35 | INFO | Using device: cuda
18:26:35 | INFO | GPU: NVIDIA GeForce RTX 3070 Laptop GPU
18:26:35 | INFO | GPU Memory: 8.6 GB
18:26:35 | INFO | Splitting dataset...
18:26:35 | INFO | Train samples: 699, Validation samples: 175
18:26:35 | INFO | Preprocessing handcrafted features...
18:26:35 | INFO | Feature preprocessing completed
18:26:35 | INFO | Creating datasets...
18:26:35 | INFO | Initializing dataset with 699 samples
18:26:35 | INFO | Initializing dataset with 175 samples
18:26:35 | INFO | DataLoader settings: batch_size=32, num_workers=4
18:26:35 | INFO | Initializing model...
18:26:35 | INFO | Initializing PokemonCNN with 18 classes and 26 handcrafted features
18:26:35 | INFO | Model parameters: 13,501,843 total, 13,501,843 trainable


TypeError: ReduceLROnPlateau.__init__() got an unexpected keyword argument 'verbose'

In [ ]:
# Plot training progress:
plot_training_history(history)

In [ ]:
# Load best model later:
model, checkpoint = load_best_model()

In [ ]:
predictions = model_inference(model, "path/to/pokemon.jpg", handcrafted_features, scaler)

In [101]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.nn.functional as F
from PIL import Image
import pandas as pd
import numpy as np
import skimage
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# Keep your original function exactly as is
def extract_image_features(image_path):
    image = skimage.io.imread(image_path)

    # Handle RGB or RGBA
    if image.shape[-1] == 4:
        rgb = image[..., :3]
        alpha = image[..., 3] / 255.0
    else:
        rgb = image
        alpha = np.ones(rgb.shape[:2])

    # Grayscale from RGB
    rgb_gray = skimage.color.rgb2gray(rgb)

    # Binary mask for region detection (ignore transparency)
    thresh = rgb_gray > 0.1
    labeled = skimage.measure.label(thresh)
    regions = skimage.measure.regionprops(labeled, intensity_image=rgb_gray)

    if len(regions) == 0:
        return {  # Return zeros with names
            'region_area': 0, 'region_perimeter': 0, 'region_eccentricity': 0,
            'region_solidity': 0, 'region_extent': 0, 'region_circularity': 0,
            'region_aspect_ratio': 0, 'region_euler_number': 0,
            'intensity_mean': 0, 'intensity_std': 0, 'intensity_min': 0, 'intensity_max': 0,
            'intensity_median': 0, 'intensity_q25': 0, 'intensity_q75': 0,
            'color_r_mean': 0, 'color_g_mean': 0, 'color_b_mean': 0,
            'color_r_std': 0, 'color_g_std': 0, 'color_b_std': 0,
            'alpha_mean': 0, 'alpha_std': 0, 'alpha_min': 0, 'alpha_max': 0
        }

    region = max(regions, key=lambda r: r.area)

    # Shape features
    shape_features = {
        'region_area': region.area,
        'region_perimeter': region.perimeter,
        'region_eccentricity': region.eccentricity,
        'region_solidity': region.solidity,
        'region_extent': region.extent,
        'region_circularity': region.perimeter / (4 * np.sqrt(region.area)) if region.area > 0 else 0,
        'region_aspect_ratio': region.major_axis_length / (region.minor_axis_length + 1e-10),
        'region_euler_number': region.euler_number
    }

    # Intensity
    coords = region.coords
    intensities = rgb_gray[coords[:, 0], coords[:, 1]]
    intensity_features = {
        'intensity_mean': intensities.mean(),
        'intensity_std': intensities.std() if len(intensities) > 1 else 0,
        'intensity_min': np.min(intensities),
        'intensity_max': np.max(intensities),
        'intensity_median': np.median(intensities),
        'intensity_q25': np.percentile(intensities, 25),
        'intensity_q75': np.percentile(intensities, 75)
    }

    # Colors
    region_mask = labeled == region.label
    region_pixels = rgb[region_mask]
    color_means = region_pixels.mean(axis=0) if region_pixels.size else [0, 0, 0]
    color_stds = region_pixels.std(axis=0) if region_pixels.size else [0, 0, 0]
    color_features = {
        'color_r_mean': color_means[0],
        'color_g_mean': color_means[1],
        'color_b_mean': color_means[2],
        'color_r_std': color_stds[0],
        'color_g_std': color_stds[1],
        'color_b_std': color_stds[2],
    }

    # Alpha
    alpha_pixels = alpha[region_mask]
    alpha_features = {
        'alpha_mean': alpha_pixels.mean() if alpha_pixels.size else 0,
        'alpha_std': alpha_pixels.std() if alpha_pixels.size else 0,
        'alpha_min': alpha_pixels.min() if alpha_pixels.size else 0,
        'alpha_max': alpha_pixels.max() if alpha_pixels.size else 0,
    }

    # Combine all
    features = {}
    features.update(shape_features)
    features.update(intensity_features)
    features.update(color_features)
    features.update(alpha_features)

    return features

# Define the CNN model
class PokemonCNN(nn.Module):
    def __init__(self, num_classes=18, num_handcrafted_features=26):
        super(PokemonCNN, self).__init__()
        
        # CNN feature extractor
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        
        self.pool = nn.MaxPool2d(2, 2)
        
        # Adaptive pooling to handle different input sizes
        self.adaptive_pool = nn.AdaptiveAvgPool2d((4, 4))
        
        # Fully connected layers for CNN features
        self.fc_cnn = nn.Sequential(
            nn.Linear(256 * 4 * 4, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(0.5)
        )
        
        # Fully connected layers for handcrafted features
        self.fc_handcrafted = nn.Sequential(
            nn.Linear(num_handcrafted_features, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.3)
        )
        
        # Combined classifier
        self.classifier = nn.Sequential(
            nn.Linear(512 + 128, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.4),
            nn.Linear(256, num_classes)
        )
    
    def forward(self, x_img, x_features):
        # CNN feature extraction
        x_img = self.pool(F.relu(self.bn1(self.conv1(x_img))))
        x_img = self.pool(F.relu(self.bn2(self.conv2(x_img))))
        x_img = self.pool(F.relu(self.bn3(self.conv3(x_img))))
        x_img = self.pool(F.relu(self.bn4(self.conv4(x_img))))
        
        x_img = self.adaptive_pool(x_img)
        x_img = torch.flatten(x_img, 1)
        x_img = self.fc_cnn(x_img)
        
        # Handcrafted features processing
        x_features = self.fc_handcrafted(x_features)
        
        # Combine features
        x = torch.cat((x_img, x_features), dim=1)
        x = self.classifier(x)
        
        return x

# Custom Dataset
class PokemonDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform
        
        # Extract feature columns
        self.feature_columns = [
            'region_area', 'region_perimeter', 'region_eccentricity', 'region_solidity',
            'region_extent', 'region_circularity', 'region_aspect_ratio', 'region_euler_number',
            'intensity_mean', 'intensity_std', 'intensity_min', 'intensity_max',
            'intensity_median', 'intensity_q25', 'intensity_q75',
            'color_r_mean', 'color_g_mean', 'color_b_mean',
            'color_r_std', 'color_g_std', 'color_b_std',
            'alpha_mean', 'alpha_std', 'alpha_min', 'alpha_max'
        ]
        
        # Target columns
        self.target_columns = [
            'bug', 'dark', 'dragon', 'electric', 'fairy', 'fighting',
            'fire', 'flying', 'ghost', 'grass', 'ground', 'ice',
            'normal', 'poison', 'psychic', 'rock', 'steel', 'water'
        ]
    
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx]['image_path']
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        # Get handcrafted features
        features = self.dataframe.iloc[idx][self.feature_columns].values.astype(np.float32)
        
        # Get targets
        targets = self.dataframe.iloc[idx][self.target_columns].values.astype(np.float32)
        
        return image, torch.tensor(features), torch.tensor(targets)

# Training function
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=25):
    best_val_loss = float('inf')
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        # Training phase
        for images, features, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
            images = images.to(device)
            features = features.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            outputs = model(images, features)
            loss = criterion(outputs, labels)
            
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * images.size(0)
            
            # Calculate accuracy
            preds = torch.sigmoid(outputs) > 0.5
            correct += (preds == labels.byte()).sum().item()
            total += labels.numel()
        
        train_loss = running_loss / len(train_loader.dataset)
        train_acc = correct / total
        
        # Validation phase
        val_loss, val_acc = evaluate_model(model, val_loader, criterion)
        
        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
        print(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")
        
        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), "models/pokemon_cnn_best_model.pt")
            print("Saved best model!")
        
        print('-' * 50)
    
    return model

# Evaluation function
def evaluate_model(model, data_loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, features, labels in data_loader:
            images = images.to(device)
            features = features.to(device)
            labels = labels.to(device)
            
            outputs = model(images, features)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item() * images.size(0)
            
            preds = torch.sigmoid(outputs) > 0.5
            correct += (preds == labels.byte()).sum().item()
            total += labels.numel()
    
    loss = running_loss / len(data_loader.dataset)
    acc = correct / total
    
    return loss, acc


In [ ]:
# Split data
train_df, val_df = train_test_split(df_pokemon_features_final2, test_size=0.2, random_state=42)
    
# Define transforms
transform = transforms.Compose([
        transforms.Resize((128, 128)),  # Resize to consistent size
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
# Create datasets
train_dataset = PokemonDataset(train_df, transform=transform)
val_dataset = PokemonDataset(val_df, transform=transform)
    
# Create dataloaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
    
# Initialize model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = PokemonCNN().to(device)
    
# Define loss and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
    
# Train the model
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=50)

Epoch 1/50 - Training:   0%|          | 0/22 [00:00<?, ?it/s]

In [95]:
# Define type classes
type_classes = ['bug','dark','dragon','electric','fairy','fighting','fire','flying',
                    'ghost','grass','ground','ice','normal','poison','psychic','rock','steel','water']
    
# Load and split data
(train_images, train_features, train_labels), \
(val_images, val_features, val_labels), \
(test_images, test_features, test_labels) = load_data_from_dataframe(df_pokemon_features_final2)
    
# Create datasets
train_dataset = PokemonDataset(train_images, train_features, train_labels)
val_dataset = PokemonDataset(val_images, val_features, val_labels)
test_dataset = PokemonDataset(test_images, test_features, test_labels)
    
# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=4)
    
# Initialize model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = PokemonCNN(num_handcrafted_features=24, num_classes=18).to(device)
    
print(f"Using device: {device}")
print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")
print(f"Test samples: {len(test_dataset)}")
    
# Train the model
print("Starting training...")
train_losses, val_losses = train_model(model, train_loader, val_loader, num_epochs=50, device=device)
    
# Load best model and evaluate
print("Loading best model for evaluation...")
model.load_state_dict(torch.load("models/pokemon_cnn_best_model.pt"))
predictions, true_labels = evaluate_model(model, test_loader, device)
    
# Calculate accuracy metrics
from sklearn.metrics import accuracy_score, classification_report
    
# Exact match accuracy (all labels must match)
exact_match_acc = accuracy_score(true_labels, predictions)
print(f"Exact match accuracy: {exact_match_acc:.4f}")
    
# Per-class accuracy
print("\nPer-class classification report:")
print(classification_report(true_labels, predictions, target_names=type_classes, zero_division=0))

Loaded 874 images successfully
Image shape: (874, 475, 475, 3)
Features shape: (874, 25)
Labels shape: (874, 18)


ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

In [96]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import numpy as np
from PIL import Image
import os
import skimage
from sklearn.preprocessing import MultiLabelBinarizer


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.nn.functional as F
from PIL import Image
import pandas as pd
import numpy as np
import skimage
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# Keep your original function exactly as is
def extract_image_features(image_path):
    image = skimage.io.imread(image_path)

    # Handle RGB or RGBA
    if image.shape[-1] == 4:
        rgb = image[..., :3]
        alpha = image[..., 3] / 255.0
    else:
        rgb = image
        alpha = np.ones(rgb.shape[:2])

    # Grayscale from RGB
    rgb_gray = skimage.color.rgb2gray(rgb)

    # Binary mask for region detection (ignore transparency)
    thresh = rgb_gray > 0.1
    labeled = skimage.measure.label(thresh)
    regions = skimage.measure.regionprops(labeled, intensity_image=rgb_gray)

    if len(regions) == 0:
        return {  # Return zeros with names
            'region_area': 0, 'region_perimeter': 0, 'region_eccentricity': 0,
            'region_solidity': 0, 'region_extent': 0, 'region_circularity': 0,
            'region_aspect_ratio': 0, 'region_euler_number': 0,
            'intensity_mean': 0, 'intensity_std': 0, 'intensity_min': 0, 'intensity_max': 0,
            'intensity_median': 0, 'intensity_q25': 0, 'intensity_q75': 0,
            'color_r_mean': 0, 'color_g_mean': 0, 'color_b_mean': 0,
            'color_r_std': 0, 'color_g_std': 0, 'color_b_std': 0,
            'alpha_mean': 0, 'alpha_std': 0, 'alpha_min': 0, 'alpha_max': 0
        }

    region = max(regions, key=lambda r: r.area)

    # Shape features
    shape_features = {
        'region_area': region.area,
        'region_perimeter': region.perimeter,
        'region_eccentricity': region.eccentricity,
        'region_solidity': region.solidity,
        'region_extent': region.extent,
        'region_circularity': region.perimeter / (4 * np.sqrt(region.area)) if region.area > 0 else 0,
        'region_aspect_ratio': region.major_axis_length / (region.minor_axis_length + 1e-10),
        'region_euler_number': region.euler_number
    }

    # Intensity
    coords = region.coords
    intensities = rgb_gray[coords[:, 0], coords[:, 1]]
    intensity_features = {
        'intensity_mean': intensities.mean(),
        'intensity_std': intensities.std() if len(intensities) > 1 else 0,
        'intensity_min': np.min(intensities),
        'intensity_max': np.max(intensities),
        'intensity_median': np.median(intensities),
        'intensity_q25': np.percentile(intensities, 25),
        'intensity_q75': np.percentile(intensities, 75)
    }

    # Colors
    region_mask = labeled == region.label
    region_pixels = rgb[region_mask]
    color_means = region_pixels.mean(axis=0) if region_pixels.size else [0, 0, 0]
    color_stds = region_pixels.std(axis=0) if region_pixels.size else [0, 0, 0]
    color_features = {
        'color_r_mean': color_means[0],
        'color_g_mean': color_means[1],
        'color_b_mean': color_means[2],
        'color_r_std': color_stds[0],
        'color_g_std': color_stds[1],
        'color_b_std': color_stds[2],
    }

    # Alpha
    alpha_pixels = alpha[region_mask]
    alpha_features = {
        'alpha_mean': alpha_pixels.mean() if alpha_pixels.size else 0,
        'alpha_std': alpha_pixels.std() if alpha_pixels.size else 0,
        'alpha_min': alpha_pixels.min() if alpha_pixels.size else 0,
        'alpha_max': alpha_pixels.max() if alpha_pixels.size else 0,
    }

    # Combine all
    features = {}
    features.update(shape_features)
    features.update(intensity_features)
    features.update(color_features)
    features.update(alpha_features)

    return features

# Define the CNN model
class PokemonCNN(nn.Module):
    def __init__(self, num_classes=18, num_handcrafted_features=26):
        super(PokemonCNN, self).__init__()
        
        # CNN feature extractor
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        
        self.pool = nn.MaxPool2d(2, 2)
        
        # Adaptive pooling to handle different input sizes
        self.adaptive_pool = nn.AdaptiveAvgPool2d((4, 4))
        
        # Fully connected layers for CNN features
        self.fc_cnn = nn.Sequential(
            nn.Linear(256 * 4 * 4, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(0.5)
        )
        
        # Fully connected layers for handcrafted features
        self.fc_handcrafted = nn.Sequential(
            nn.Linear(num_handcrafted_features, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.3)
        )
        
        # Combined classifier
        self.classifier = nn.Sequential(
            nn.Linear(512 + 128, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.4),
            nn.Linear(256, num_classes)
        )
    
    def forward(self, x_img, x_features):
        # CNN feature extraction
        x_img = self.pool(F.relu(self.bn1(self.conv1(x_img))))
        x_img = self.pool(F.relu(self.bn2(self.conv2(x_img))))
        x_img = self.pool(F.relu(self.bn3(self.conv3(x_img))))
        x_img = self.pool(F.relu(self.bn4(self.conv4(x_img))))
        
        x_img = self.adaptive_pool(x_img)
        x_img = torch.flatten(x_img, 1)
        x_img = self.fc_cnn(x_img)
        
        # Handcrafted features processing
        x_features = self.fc_handcrafted(x_features)
        
        # Combine features
        x = torch.cat((x_img, x_features), dim=1)
        x = self.classifier(x)
        
        return x

# Custom Dataset
class PokemonDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform
        
        # Extract feature columns
        self.feature_columns = [
            'region_area', 'region_perimeter', 'region_eccentricity', 'region_solidity',
            'region_extent', 'region_circularity', 'region_aspect_ratio', 'region_euler_number',
            'intensity_mean', 'intensity_std', 'intensity_min', 'intensity_max',
            'intensity_median', 'intensity_q25', 'intensity_q75',
            'color_r_mean', 'color_g_mean', 'color_b_mean',
            'color_r_std', 'color_g_std', 'color_b_std',
            'alpha_mean', 'alpha_std', 'alpha_min', 'alpha_max'
        ]
        
        # Target columns
        self.target_columns = [
            'bug', 'dark', 'dragon', 'electric', 'fairy', 'fighting',
            'fire', 'flying', 'ghost', 'grass', 'ground', 'ice',
            'normal', 'poison', 'psychic', 'rock', 'steel', 'water'
        ]
    
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx]['image_path']
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        # Get handcrafted features
        features = self.dataframe.iloc[idx][self.feature_columns].values.astype(np.float32)
        
        # Get targets
        targets = self.dataframe.iloc[idx][self.target_columns].values.astype(np.float32)
        
        return image, torch.tensor(features), torch.tensor(targets)

# Training function
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=25):
    best_val_loss = float('inf')
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        # Training phase
        for images, features, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
            images = images.to(device)
            features = features.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            outputs = model(images, features)
            loss = criterion(outputs, labels)
            
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * images.size(0)
            
            # Calculate accuracy
            preds = torch.sigmoid(outputs) > 0.5
            correct += (preds == labels.byte()).sum().item()
            total += labels.numel()
        
        train_loss = running_loss / len(train_loader.dataset)
        train_acc = correct / total
        
        # Validation phase
        val_loss, val_acc = evaluate_model(model, val_loader, criterion)
        
        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
        print(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")
        
        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), "models/pokemon_cnn_best_model.pt")
            print("Saved best model!")
        
        print('-' * 50)
    
    return model

# Evaluation function
def evaluate_model(model, data_loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, features, labels in data_loader:
            images = images.to(device)
            features = features.to(device)
            labels = labels.to(device)
            
            outputs = model(images, features)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item() * images.size(0)
            
            preds = torch.sigmoid(outputs) > 0.5
            correct += (preds == labels.byte()).sum().item()
            total += labels.numel()
    
    loss = running_loss / len(data_loader.dataset)
    acc = correct / total
    
    return loss, acc

In [ ]:
# Split data
train_df, val_df = train_test_split(df_pokemon_features_final2, test_size=0.2, random_state=42)
    
# Define transforms
transform = transforms.Compose([
        transforms.Resize((128, 128)),  # Resize to consistent size
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
# Create datasets
train_dataset = PokemonDataset(train_df, transform=transform)
val_dataset = PokemonDataset(val_df, transform=transform)
    
# Create dataloaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
    
# Initialize model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = PokemonCNN().to(device)
    
# Define loss and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
    
# Train the model
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=50)

In [ ]:
# Define type classes
type_classes = ['bug','dark','dragon','electric','fairy','fighting','fire','flying',
                    'ghost','grass','ground','ice','normal','poison','psychic','rock','steel','water']
    
# Initialize model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = PokemonCNN(num_handcrafted_features=25, num_classes=18).to(device)
    
print(f"Using device: {device}")
print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")
    
# Example data preparation (replace with your actual data)
# images: numpy array of shape (N, 475, 475, 3)
# handcrafted_features: numpy array of shape (N, 24) - from your extract_image_features function
# labels: numpy array of shape (N, 18) - multi-hot encoded labels
    
# Create datasets and dataloaders
# train_dataset = PokemonDataset(train_images, train_features, train_labels)
# val_dataset = PokemonDataset(val_images, val_features, val_labels)
# test_dataset = PokemonDataset(test_images, test_features, test_labels)
    
# train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
# test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)
    
# Train the model
# train_losses, val_losses = train_model(model, train_loader, val_loader, num_epochs=50, device=device)
    
# Load best model and evaluate
# model.load_state_dict(torch.load("models/pokemon_cnn_best_model.pt"))
# predictions, true_labels = evaluate_model(model, test_loader, device)
    
print("Model architecture created successfully!")
print("To use this model:")
print("1. Prepare your data: images (N, 475, 475, 3), handcrafted features (N, 24), labels (N, 18)")
print("2. Create datasets and dataloaders")
print("3. Call train_model() to train the CNN")
print("4. The best model will be automatically saved to 'models/pokemon_cnn_best_model.pt'")

In [30]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class PokemonCNN(nn.Module):
    def __init__(self, num_classes, num_features=25, image_input_channels=3):
        super(PokemonCNN, self).__init__()

        self.cnn = nn.Sequential(
            nn.Conv2d(image_input_channels, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),  # 112x112

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),  # 56x56

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),  # 28x28

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d((1, 1))  # Global Average Pooling → (B, 256, 1, 1)
        )

        # CNN output flattened is (B, 256)
        self.cnn_fc = nn.Sequential(
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3)
        )

        # Features from extract_image_features (25 features)
        self.feature_fc = nn.Sequential(
            nn.Linear(num_features, 64),
            nn.ReLU(),
            nn.Dropout(0.3)
        )

        # Final classifier
        self.classifier = nn.Sequential(
            nn.Linear(128 + 64, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, num_classes),
            nn.Sigmoid()  # Multi-label output
        )

    def forward(self, image, features):
        x = self.cnn(image)
        x = x.view(x.size(0), -1)  # Flatten from (B, 256, 1, 1) → (B, 256)
        x = self.cnn_fc(x)

        f = self.feature_fc(features)

        combined = torch.cat([x, f], dim=1)
        return self.classifier(combined)


In [31]:
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

def train_pokemon_cnn(model, images_tensor, features_tensor, labels_tensor,
                      num_epochs=30, batch_size=32, lr=1e-3):

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    X_img_train, X_img_val, X_feat_train, X_feat_val, y_train, y_val = train_test_split(
        images_tensor, features_tensor, labels_tensor, test_size=0.2, random_state=42)

    train_ds = TensorDataset(X_img_train, X_feat_train, y_train)
    val_ds = TensorDataset(X_img_val, X_feat_val, y_val)

    train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    val_dl = DataLoader(val_ds, batch_size=batch_size)

    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for xb_img, xb_feat, yb in train_dl:
            xb_img, xb_feat, yb = xb_img.to(device), xb_feat.to(device), yb.to(device)
            preds = model(xb_img, xb_feat)
            loss = criterion(preds, yb.float())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for xb_img, xb_feat, yb in val_dl:
                xb_img, xb_feat, yb = xb_img.to(device), xb_feat.to(device), yb.to(device)
                preds = model(xb_img, xb_feat)
                val_loss += criterion(preds, yb.float()).item()

        print(f"Epoch {epoch+1}/{num_epochs} — Train Loss: {total_loss/len(train_dl):.4f} — Val Loss: {val_loss/len(val_dl):.4f}")


In [39]:
# List of features (exclude non-feature columns)
feature_cols = [
    'region_area', 'region_perimeter', 'region_eccentricity', 'region_solidity',
    'region_extent', 'region_circularity', 'region_aspect_ratio', 'region_euler_number',
    'intensity_mean', 'intensity_std', 'intensity_min', 'intensity_max', 'intensity_median',
    'intensity_q25', 'intensity_q75', 'color_r_mean', 'color_g_mean', 'color_b_mean',
    'color_r_std', 'color_g_std', 'color_b_std', 'alpha_mean', 'alpha_std', 'alpha_min', 'alpha_max'
]

label_cols = [
    'bug','dark','dragon','electric','fairy','fighting','fire','flying','ghost','grass',
    'ground','ice','normal','poison','psychic','rock','steel','water'
]


In [40]:
# Features
X_features = df_pokemon_features_final2[feature_cols].values
scaler = StandardScaler()
X_features = scaler.fit_transform(X_features)

# Labels
y = df_pokemon_features_final2[label_cols].values

# Convert to tensors
X_features_tensor = torch.tensor(X_features, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

# Dummy image tensor (since you already extracted features)
# shape = (N, 3, 224, 224) filled with zeros (for now)
X_images_tensor = torch.zeros((len(df_pokemon_features_final2), 3, 224, 224), dtype=torch.float32)


In [41]:
num_classes = y_tensor.shape[1]
model = PokemonCNN(num_classes=num_classes, num_features=len(feature_cols))

train_pokemon_cnn(
    model=model,
    images_tensor=X_images_tensor,
    features_tensor=X_features_tensor,
    labels_tensor=y_tensor,
    num_epochs=1000,
    batch_size=32,
    lr=1e-3
)


Epoch 1/1000 — Train Loss: 0.4586 — Val Loss: 0.3643
Epoch 2/1000 — Train Loss: 0.3017 — Val Loss: 0.2985
Epoch 3/1000 — Train Loss: 0.2960 — Val Loss: 0.2836
Epoch 4/1000 — Train Loss: 0.2936 — Val Loss: 0.5222
Epoch 5/1000 — Train Loss: 0.2939 — Val Loss: 1.1763
Epoch 6/1000 — Train Loss: 0.2924 — Val Loss: 1.3888
Epoch 7/1000 — Train Loss: 0.2912 — Val Loss: 1.5832
Epoch 8/1000 — Train Loss: 0.2904 — Val Loss: 1.4516
Epoch 9/1000 — Train Loss: 0.2903 — Val Loss: 0.3951
Epoch 10/1000 — Train Loss: 0.2878 — Val Loss: 0.5280
Epoch 11/1000 — Train Loss: 0.2872 — Val Loss: 5.6737
Epoch 12/1000 — Train Loss: 0.2871 — Val Loss: 4.9258
Epoch 13/1000 — Train Loss: 0.2858 — Val Loss: 1.2573
Epoch 14/1000 — Train Loss: 0.2853 — Val Loss: 1.1773
Epoch 15/1000 — Train Loss: 0.2852 — Val Loss: 0.9001
Epoch 16/1000 — Train Loss: 0.2835 — Val Loss: 0.6806
Epoch 17/1000 — Train Loss: 0.2835 — Val Loss: 1.2626
Epoch 18/1000 — Train Loss: 0.2810 — Val Loss: 0.5726
Epoch 19/1000 — Train Loss: 0.2813 — 

In [42]:
torch.save(model.state_dict(), "models/pokemon_cnn_best_model.pt")


In [80]:
pikachu_teste = pd.read_csv("test_set/pokemon_image_features_test.csv")


In [81]:
pikachu_teste =  pikachu_teste[pikachu_teste['id'] == 116]

In [82]:
pikachu_teste.head()

,image_path,id,region_area,region_perimeter,region_eccentricity,region_solidity,region_extent,region_circularity,region_aspect_ratio,region_euler_number,...,color_r_mean,color_g_mean,color_b_mean,color_r_std,color_g_std,color_b_std,alpha_mean,alpha_std,alpha_min,alpha_max
54,test_set\horsea_116.png,116,71752.0,4152.732465,0.854769,0.667355,0.493073,3.875761,1.926748,-216,...,138.001045,197.348311,215.179883,46.723803,33.955399,36.022668,0.999999,0.000133,0.964706,1.0


In [49]:
from torchvision import transforms
from PIL import Image
import torch

def preprocess_image(image_path, image_size=(224, 224)):
    transform = transforms.Compose([
        transforms.Resize(image_size),
        transforms.ToTensor(),  # Converts to [C, H, W] and scales to [0,1]
    ])
    img = Image.open(image_path).convert('RGB')  # Drop alpha for CNN
    return transform(img).unsqueeze(0)  # Add batch dimension


In [71]:
def predict_pokemon_type(model, image_path, type_classes, df_features, device='cuda'):
    import numpy as np
    
    model.eval()
    device = torch.device(device if torch.cuda.is_available() else 'cpu')
    model = model.to(device)

    # Preprocess image
    image_tensor = preprocess_image(image_path).to(device)

    # ✅ Convert features to a clean float32 tensor
    if isinstance(df_features, pd.DataFrame):
        features = df_features.iloc[0].astype(float).values
    elif isinstance(df_features, pd.Series):
        features = df_features.astype(float).values
    else:
        features = np.array(df_features, dtype=np.float32)

    features_tensor = torch.tensor(features, dtype=torch.float32).unsqueeze(0).to(device)

    # Forward pass
    with torch.no_grad():
        output = model(image_tensor, features_tensor)
        probabilities = torch.sigmoid(output).squeeze()

    # Threshold for multi-label prediction
    predicted_types = [type_classes[i] for i, prob in enumerate(probabilities) if prob > 0.5]

    return predicted_types, probabilities.cpu().numpy()


In [83]:
type_classes = ['bug','dark','dragon','electric','fairy','fighting','fire','flying',
                'ghost','grass','ground','ice','normal','poison','psychic','rock','steel','water']


model = PokemonCNN(num_classes=len(type_classes), num_features=25)
model.load_state_dict(torch.load("models/pokemon_cnn_best_model.pt"))


<All keys matched successfully>

In [84]:
exclude_cols = ['image_path', 'id']
selected_feature_cols = [col for col in pikachu_teste.columns if col not in exclude_cols]

pikachu_teste = pikachu_teste[selected_feature_cols]


In [85]:
pikachu_teste.head()

,region_area,region_perimeter,region_eccentricity,region_solidity,region_extent,region_circularity,region_aspect_ratio,region_euler_number,intensity_mean,intensity_std,...,color_r_mean,color_g_mean,color_b_mean,color_r_std,color_g_std,color_b_std,alpha_mean,alpha_std,alpha_min,alpha_max
54,71752.0,4152.732465,0.854769,0.667355,0.493073,3.875761,1.926748,-216,0.729501,0.13153,...,138.001045,197.348311,215.179883,46.723803,33.955399,36.022668,0.999999,0.000133,0.964706,1.0


In [86]:
predicted_types, probs = predict_pokemon_type(
    model, "test_set/pikachu_25.png", type_classes, pikachu_teste
)

print("🔮 Predicted Types:", predicted_types)


🔮 Predicted Types: ['fire', 'ground']


In [24]:
# Set MLflow tracking URI
mlflow.set_tracking_uri("http://127.0.0.1:5000/")

def load_and_preprocess_data(csv_path):
    """Load and preprocess the Pokemon dataset"""
    df = pd.read_csv(csv_path)
    
    # Remove unwanted columns
    feature_cols = [col for col in df.columns if col not in ['image_path', 'id', 'name', 'types']]
    X = df[feature_cols].values
    
    # Handle multi-label types (take first type for simplification)
    df['primary_type'] = df['types'].str.split(',').str[0].str.strip()
    y = df['primary_type'].values
    
    # Encode labels
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    
    # Standardize features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    return X_scaled, y_encoded, label_encoder, scaler, feature_cols

def train_model(model, train_loader, val_loader, num_epochs=100, learning_rate=0.001):
    """Train the Pokemon CNN model"""
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=10, factor=0.5)
    
    best_val_loss = float('inf')
    best_model_state = None
    
    try:
        # Start MLflow run
        with mlflow.start_run(run_name="Pokemon_CNN_Training"):
            # Log hyperparameters
            mlflow.log_param("epochs", num_epochs)
            mlflow.log_param("learning_rate", learning_rate)
            mlflow.log_param("batch_size", train_loader.batch_size)
            mlflow.log_param("optimizer", "Adam")
            mlflow.log_param("device", str(device))
            
            for epoch in range(num_epochs):
                # Training phase
                model.train()
                train_loss = 0.0
                train_correct = 0
                train_total = 0
                
                for features, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}'):
                    features, labels = features.to(device), labels.to(device)
                    
                    optimizer.zero_grad()
                    outputs = model(features)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()
                    
                    train_loss += loss.item()
                    _, predicted = torch.max(outputs.data, 1)
                    train_total += labels.size(0)
                    train_correct += (predicted == labels).sum().item()
                
                # Validation phase
                model.eval()
                val_loss = 0.0
                val_correct = 0
                val_total = 0
                
                with torch.no_grad():
                    for features, labels in val_loader:
                        features, labels = features.to(device), labels.to(device)
                        outputs = model(features)
                        loss = criterion(outputs, labels)
                        
                        val_loss += loss.item()
                        _, predicted = torch.max(outputs.data, 1)
                        val_total += labels.size(0)
                        val_correct += (predicted == labels).sum().item()
                
                # Calculate metrics
                train_loss /= len(train_loader)
                val_loss /= len(val_loader)
                train_acc = 100 * train_correct / train_total
                val_acc = 100 * val_correct / val_total
                
                # Update learning rate
                scheduler.step(val_loss)
                
                # Save best model
                if val_loss < best_val_loss:
                    best_val_loss = val_loss
                    best_model_state = model.state_dict().copy()
                
                # Log metrics to MLflow
                try:
                    mlflow.log_metric("train_loss", train_loss, step=epoch)
                    mlflow.log_metric("val_loss", val_loss, step=epoch)
                    mlflow.log_metric("train_accuracy", train_acc, step=epoch)
                    mlflow.log_metric("val_accuracy", val_acc, step=epoch)
                    mlflow.log_metric("learning_rate", optimizer.param_groups[0]['lr'], step=epoch)
                except Exception as e:
                    logger.warning(f"MLflow logging error (skipping): {e}")
                
                
                logger.info(f'Epoch [{epoch+1}/{num_epochs}]')
                
                logger.info(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%')
                
                logger.info(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')

                logger.info('-' * 50)
            
            # Load best model
            if best_model_state:
                model.load_state_dict(best_model_state)
            
            # Log model to MLflow
            try:
                mlflow.pytorch.log_model(model, "pokemon_cnn_model")
                logger.success("Model logged to MLflow successfully!")
            except Exception as e:
                logger.error(f"MLflow model logging error (skipping): {e}")
    
    except Exception as e:
        logger.warning(f"MLflow run error (continuing without logging): {e}")
    
    return model

In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import mlflow
import mlflow.pytorch
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Set MLflow tracking URI
mlflow.set_tracking_uri("http://127.0.0.1:5000/")

def load_and_preprocess_data(df):
    """Load and preprocess the Pokemon dataframe"""
    # Remove unwanted columns
    feature_cols = [col for col in df.columns if col not in ['image_path', 'id', 'name', 'types']]
    X = df[feature_cols].values
    
    # Handle multi-label types (take first type for simplification)
    df['primary_type'] = df['types'].str.split(',').str[0].str.strip()
    y = df['primary_type'].values
    
    # Encode labels
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    
    # Standardize features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    return X_scaled, y_encoded, label_encoder, scaler, feature_cols

def train_model(model, X_train, y_train, X_val, y_val, num_epochs=100, learning_rate=0.001, batch_size=32):
    """Train the Pokemon CNN model"""
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    # Convert to tensors
    X_train_tensor = torch.FloatTensor(X_train)
    y_train_tensor = torch.LongTensor(y_train)
    X_val_tensor = torch.FloatTensor(X_val)
    y_val_tensor = torch.LongTensor(y_val)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=10, factor=0.5)
    
    best_val_loss = float('inf')
    best_model_state = None
    
    # Calculate number of batches
    n_train_samples = X_train_tensor.size(0)
    n_batches = (n_train_samples + batch_size - 1) // batch_size
    
    try:
        # Start MLflow run
        with mlflow.start_run(run_name="Pokemon_CNN_Training"):
            # Log hyperparameters
            mlflow.log_param("epochs", num_epochs)
            mlflow.log_param("learning_rate", learning_rate)
            mlflow.log_param("batch_size", batch_size)
            mlflow.log_param("optimizer", "Adam")
            mlflow.log_param("device", str(device))
            
            for epoch in range(num_epochs):
                # Training phase
                model.train()
                train_loss = 0.0
                train_correct = 0
                train_total = 0
                
                # Create random indices for batching
                indices = torch.randperm(n_train_samples)
                
                for i in tqdm(range(n_batches), desc=f'Epoch {epoch+1}/{num_epochs}'):
                    start_idx = i * batch_size
                    end_idx = min((i + 1) * batch_size, n_train_samples)
                    batch_indices = indices[start_idx:end_idx]
                    
                    features = X_train_tensor[batch_indices].to(device)
                    labels = y_train_tensor[batch_indices].to(device)
                    
                    optimizer.zero_grad()
                    outputs = model(features)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()
                    
                    train_loss += loss.item()
                    _, predicted = torch.max(outputs.data, 1)
                    train_total += labels.size(0)
                    train_correct += (predicted == labels).sum().item()
                
                # Validation phase
                model.eval()
                with torch.no_grad():
                    X_val_device = X_val_tensor.to(device)
                    y_val_device = y_val_tensor.to(device)
                    val_outputs = model(X_val_device)
                    val_loss = criterion(val_outputs, y_val_device).item()
                    
                    _, val_predicted = torch.max(val_outputs.data, 1)
                    val_correct = (val_predicted == y_val_device).sum().item()
                    val_total = y_val_device.size(0)
                
                # Calculate metrics
                train_loss /= n_batches
                train_acc = 100 * train_correct / train_total
                val_acc = 100 * val_correct / val_total
                
                # Update learning rate
                scheduler.step(val_loss)
                
                # Save best model
                if val_loss < best_val_loss:
                    best_val_loss = val_loss
                    best_model_state = model.state_dict().copy()
                
                # Log metrics to MLflow
                try:
                    mlflow.log_metric("train_loss", train_loss, step=epoch)
                    mlflow.log_metric("val_loss", val_loss, step=epoch)
                    mlflow.log_metric("train_accuracy", train_acc, step=epoch)
                    mlflow.log_metric("val_accuracy", val_acc, step=epoch)
                    mlflow.log_metric("learning_rate", optimizer.param_groups[0]['lr'], step=epoch)
                except Exception as e:
                    print(f"MLflow logging error (skipping): {e}")
                
                print(f'Epoch [{epoch+1}/{num_epochs}]')
                print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%')
                print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')
                print('-' * 50)
            
            # Load best model
            if best_model_state:
                model.load_state_dict(best_model_state)
            
            # Log model to MLflow
            try:
                mlflow.pytorch.log_model(model, "pokemon_cnn_model")
                print("Model logged to MLflow successfully!")
            except Exception as e:
                print(f"MLflow model logging error (skipping): {e}")
    
    except Exception as e:
        print(f"MLflow run error (continuing without logging): {e}")
    
    return model

def main(df):
    # Load and preprocess data
    print("Loading and preprocessing data...")
    X, y, label_encoder, scaler, feature_cols = load_and_preprocess_data(df)
    
    print(f"Dataset shape: {X.shape}")
    print(f"Number of classes: {len(np.unique(y))}")
    print(f"Feature columns: {feature_cols}")
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    
    X_train, X_val, y_train, y_val = train_test_split(
        X_train, y_train, test_size=0.2, random_state=42, stratify=y_train
    )
    
    # Initialize model
    num_classes = len(np.unique(y))
    model = PokemonCNN(input_dim=X.shape[1], num_classes=num_classes)
    
    print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")
    
    # Train model
    print("Starting training...")
    trained_model = train_model(model, X_train, y_train, X_val, y_val, num_epochs=100)
    
    # Test model
    print("Testing model...")
    trained_model.eval()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    trained_model.to(device)
    
    with torch.no_grad():
        X_test_tensor = torch.FloatTensor(X_test).to(device)
        y_test_tensor = torch.LongTensor(y_test).to(device)
        test_outputs = trained_model(X_test_tensor)
        _, test_predictions = torch.max(test_outputs.data, 1)
        
        test_predictions = test_predictions.cpu().numpy()
        test_labels = y_test
    
    # Calculate test accuracy
    test_accuracy = accuracy_score(test_labels, test_predictions)
    print(f"\nTest Accuracy: {test_accuracy:.4f}")
    
    # Print classification report
    print("\nClassification Report:")
    print(classification_report(test_labels, test_predictions, 
                              target_names=label_encoder.classes_))
    
    # Save model and preprocessors
    torch.save({
        'model_state_dict': trained_model.state_dict(),
        'label_encoder': label_encoder,
        'scaler': scaler,
        'feature_cols': feature_cols,
        'num_classes': num_classes,
        'input_dim': X.shape[1]
    }, 'pokemon_cnn_complete.pth')
    
    print("Model and preprocessors saved to 'pokemon_cnn_complete.pth'")

if __name__ == "__main__":
    # Load your dataframe here
    main(df_pokemon_features_final)

Loading and preprocessing data...
Dataset shape: (874, 25)
Number of classes: 18
Feature columns: ['region_area', 'region_perimeter', 'region_eccentricity', 'region_solidity', 'region_extent', 'region_circularity', 'region_aspect_ratio', 'region_euler_number', 'intensity_mean', 'intensity_std', 'intensity_min', 'intensity_max', 'intensity_median', 'intensity_q25', 'intensity_q75', 'color_r_mean', 'color_g_mean', 'color_b_mean', 'color_r_std', 'color_g_std', 'color_b_std', 'alpha_mean', 'alpha_std', 'alpha_min', 'alpha_max']
Model parameters: 3,960,530
Starting training...


Epoch 1/100: 100%|██████████| 18/18 [00:01<00:00, 13.74it/s]


Epoch [1/100]
Train Loss: 2.8392, Train Acc: 10.02%
Val Loss: 2.9849, Val Acc: 12.14%
--------------------------------------------------


Epoch 2/100: 100%|██████████| 18/18 [00:00<00:00, 64.43it/s]


Epoch [2/100]
Train Loss: 2.7794, Train Acc: 10.55%
Val Loss: 2.7894, Val Acc: 12.14%
--------------------------------------------------


...

Epoch [99/100]
Train Loss: 1.6840, Train Acc: 40.79%
Val Loss: 5.0360, Val Acc: 12.14%
--------------------------------------------------


Epoch 100/100: 100%|██████████| 18/18 [00:00<00:00, 79.06it/s]
2025/06/25 03:37:42 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Epoch [100/100]
Train Loss: 1.6422, Train Acc: 42.58%
Val Loss: 4.9469, Val Acc: 12.86%
--------------------------------------------------


2025/06/25 03:37:42 WARNING mlflow.utils.requirements_utils: Found torch version (2.7.0+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torch==2.7.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/06/25 03:37:55 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Model logged to MLflow successfully!
🏃 View run Pokemon_CNN_Training at: http://127.0.0.1:5000/#/experiments/0/runs/405beede2c4f4cc79644c9d22c18d59f
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/0
Testing model...

Test Accuracy: 0.1543

Classification Report:
              precision    recall  f1-score   support

         bug       0.12      0.14      0.13        14
        dark       0.00      0.00      0.00         9
      dragon       0.00      0.00      0.00         7
    electric       0.18      0.30      0.22        10
       fairy       0.00      0.00      0.00         5
    fighting       0.33      0.14      0.20         7
        fire       0.25      0.18      0.21        11
      flying       0.00      0.00      0.00         2
       ghost       0.00      0.00      0.00         6
       grass       0.15      0.11      0.13        18
      ground       0.00      0.00      0.00         6
         ice       0.33      0.17      0.22         6
      normal       0.14 

In [ ]:

def main():
    # Load and preprocess data
    print("Loading and preprocessing data...")
    X, y, label_encoder, scaler, feature_cols = load_and_preprocess_data('pokemon_features.csv')
    
    print(f"Dataset shape: {X.shape}")
    print(f"Number of classes: {len(np.unique(y))}")
    print(f"Feature columns: {feature_cols}")
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    
    X_train, X_val, y_train, y_val = train_test_split(
        X_train, y_train, test_size=0.2, random_state=42, stratify=y_train
    )
    
    # Create datasets and dataloaders
    train_dataset = PokemonDataset(X_train, y_train)
    val_dataset = PokemonDataset(X_val, y_val)
    test_dataset = PokemonDataset(X_test, y_test)
    
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
    
    # Initialize model
    num_classes = len(np.unique(y))
    model = PokemonCNN(input_dim=X.shape[1], num_classes=num_classes)
    
    print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")
    
    # Train model
    print("Starting training...")
    trained_model = train_model(model, train_loader, val_loader, num_epochs=100)
    
    # Test model
    print("Testing model...")
    trained_model.eval()
    test_predictions = []
    test_labels = []
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    trained_model.to(device)
    
    with torch.no_grad():
        for features, labels in test_loader:
            features, labels = features.to(device), labels.to(device)
            outputs = trained_model(features)
            _, predicted = torch.max(outputs.data, 1)
            
            test_predictions.extend(predicted.cpu().numpy())
            test_labels.extend(labels.cpu().numpy())
    
    # Calculate test accuracy
    test_accuracy = accuracy_score(test_labels, test_predictions)
    print(f"\nTest Accuracy: {test_accuracy:.4f}")
    
    # Print classification report
    print("\nClassification Report:")
    print(classification_report(test_labels, test_predictions, 
                              target_names=label_encoder.classes_))
    
    # Save model and preprocessors
    torch.save({
        'model_state_dict': trained_model.state_dict(),
        'label_encoder': label_encoder,
        'scaler': scaler,
        'feature_cols': feature_cols,
        'num_classes': num_classes,
        'input_dim': X.shape[1]
    }, 'pokemon_cnn_complete.pth')
    
    print("Model and preprocessors saved to 'pokemon_cnn_complete.pth'")

if __name__ == "__main__":
    main()

In [17]:
def predict_pokemon_type(model, le, features: list, device="cuda" if torch.cuda.is_available() else "cpu"):
    model.eval()
    x = torch.tensor([features], dtype=torch.float32).to(device)
    with torch.no_grad():
        logits = model(x)
        pred_idx = torch.argmax(logits, dim=1).item()
    return le.inverse_transform([pred_idx])[0]


In [19]:
df_pokemon_features_final.head()

,image_path,id,region_area,region_perimeter,region_eccentricity,region_solidity,region_extent,region_circularity,region_aspect_ratio,region_euler_number,...,color_b_mean,color_r_std,color_g_std,color_b_std,alpha_mean,alpha_std,alpha_min,alpha_max,name,types
0,train_set\abomasnow_460.png,460,107151.0,7448.593826,0.371940,0.685367,0.584940,5.688741,1.077288,-946,...,194.553807,70.389289,58.600939,58.962556,0.999971,0.001373,0.862745,1.0,Abomasnow,"grass, ice"
1,train_set\absol_359.png,359,151465.0,7464.088850,0.484258,0.671313,0.671313,4.794691,1.142955,-347,...,241.274413,47.515171,43.850289,36.178825,0.329138,0.469837,0.000000,1.0,Absol,dark
2,train_set\accelgor_617.png,617,122602.0,8408.956995,0.310894,0.543388,0.543388,6.003899,1.052139,-552,...,190.481844,72.049311,72.536374,67.004721,0.528052,0.499164,0.000000,1.0,Accelgor,bug
3,train_set\aegislash-shield_681.png,681,71327.0,3475.656854,0.534266,0.316131,0.316131,3.253493,1.182989,0,...,255.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,Aegislash-Shield,"steel, ghost"
4,train_set\aggron_306.png,306,147550.0,15195.167779,0.487770,0.653961,0.653961,9.889536,1.145512,-807,...,169.569143,76.058984,75.476941,73.497089,0.725863,0.446061,0.000000,1.0,Aggron,"steel, rock"


In [18]:
model, label_encoder, best_model_path = train_pokemon_model(df_pokemon_features_final, model_name="PokemonTypeClassifier")




ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

In [ ]:
# Later: Load model
loaded_model = PokemonMLP(input_dim=25, num_classes=len(label_encoder.classes_))
loaded_model.load_state_dict(torch.load(best_model_path))
loaded_model.eval()



Agora vamos obter nosso conjunto de teste 


In [89]:
show_pokemon(25, width=400, max_moves=8)

Hp,35
Attack,55
Defense,40
Special-Attack,50
Special-Defense,50
Speed,90


In [ ]:
df_pokemon_test = collect_pokemon_data(1, 151)

In [ ]:
df_pokemon_test.head()

,id,name,species,weight,types,abilities,image_url,moves,stat_hp,stat_attack,stat_defense,stat_special-attack,stat_special-defense,stat_speed
0,1,Bulbasaur,bulbasaur,69,"grass, poison","overgrow, chlorophyll",https://raw.githubusercontent.com/PokeAPI/spri...,"razor-wind, swords-dance, cut, bind, vine-whip...",45,49,49,65,65,45
1,2,Ivysaur,ivysaur,130,"grass, poison","overgrow, chlorophyll",https://raw.githubusercontent.com/PokeAPI/spri...,"swords-dance, cut, bind, vine-whip, headbutt, ...",60,62,63,80,80,60
2,3,Venusaur,venusaur,1000,"grass, poison","overgrow, chlorophyll",https://raw.githubusercontent.com/PokeAPI/spri...,"swords-dance, cut, bind, vine-whip, headbutt, ...",80,82,83,100,100,80
3,4,Charmander,charmander,85,fire,"blaze, solar-power",https://raw.githubusercontent.com/PokeAPI/spri...,"mega-punch, fire-punch, thunder-punch, scratch...",39,52,43,60,50,65
4,5,Charmeleon,charmeleon,190,fire,"blaze, solar-power",https://raw.githubusercontent.com/PokeAPI/spri...,"mega-punch, fire-punch, thunder-punch, scratch...",58,64,58,80,65,80


In [ ]:
df_pokemon_test.to_csv('test_set/pokemon_data_test.csv', index=False)
download_pokemon_images(df_pokemon_test, output_dir='test_set')

Maravilha, vamos então realizar a inferênmcia utilizando nosso modelo e comparando o resultado da previsão 

In [46]:
df_pokemon_features_test = process_images_in_folder('test_set')
df_pokemon_features_test.to_csv("test_set/pokemon_image_features_test.csv", index=False)


Processing images:   0%|          | 0/151 [00:00<?, ?it/s]2025-06-25 14:49:50.755 | INFO     | __main__:process_images_in_folder:26 - ✅ Processed abra_63.png
2025-06-25 14:49:50.818 | INFO     | __main__:process_images_in_folder:26 - ✅ Processed aerodactyl_142.png
Processing images:   1%|▏         | 2/151 [00:00<00:09, 15.66it/s]2025-06-25 14:49:50.874 | INFO     | __main__:process_images_in_folder:26 - ✅ Processed alakazam_65.png
2025-06-25 14:49:50.952 | INFO     | __main__:process_images_in_folder:26 - ✅ Processed arbok_24.png
Processing images:   3%|▎         | 4/151 [00:00<00:09, 15.21it/s]2025-06-25 14:49:51.026 | INFO     | __main__:process_images_in_folder:26 - ✅ Processed arcanine_59.png
2025-06-25 14:49:51.087 | INFO     | __main__:process_images_in_folder:26 - ✅ Processed articuno_144.png
Processing images:   4%|▍         | 6/151 [00:00<00:09, 15.03it/s]2025-06-25 14:49:51.157 | INFO     | __main__:process_images_in_folder:26 - ✅ Processed beedrill_15.png
2025-06-25 14:49:51

Agora vamos avaliar os resultados utilizando algumas métricas como a matriz de confusão